<a href="https://colab.research.google.com/github/tokaalaa/MRNet/blob/master/OR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Imports

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from ipywidgets import interact, Dropdown, IntSlider
from tqdm import tqdm_notebook
from __future__ import division

import tensorflow as tf
from tensorflow import keras
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import LSTM
from tensorflow.keras import models
from tensorflow.keras import layers

##Load Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "drive/My Drive/Colab Notebooks/MRNet-v1.0"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/MRNet-v1.0


In [ ]:
#Data generator
class data_generator(keras.utils.Sequence):
    def __init__(self, injury, plane, data_type):
        self.injury = injury
        self.plane = plane
        self.data_type = data_type
        if(data_type == 'train'):
          self.labels = pd.read_csv('train-{}.csv'.format(self.injury), names=['case', 'label'], header=None,
                              dtype={'case': str, 'label': np.int64})['label'].tolist()
          self.case_list = pd.read_csv('train-{}.csv'.format(self.injury), names=['case', 'label'], header=None,
                               dtype={'case': str, 'label': np.int64})['case'].tolist()
          self.labels = self.labels[0:1017]
          self.case_list = self.case_list[0:1017]
          self.data_path = 'train'
        elif(data_type == 'valid'):
          self.labels = pd.read_csv('train-{}.csv'.format(self.injury), names=['case', 'label'], header=None,
                              dtype={'case': str, 'label': np.int64})['label'].tolist()
          self.case_list = pd.read_csv('train-{}.csv'.format(self.injury), names=['case', 'label'], header=None,
                               dtype={'case': str, 'label': np.int64})['case'].tolist()
          self.labels = self.labels[1017:1130]
          self.case_list = self.case_list[1017:1130]
          self.data_path = 'train'
        else:
          self.labels = pd.read_csv('valid-{}.csv'.format(self.injury), names=['case', 'label'], header=None,
                              dtype={'case': str, 'label': np.int64})['label'].tolist()
          self.case_list = pd.read_csv('valid-{}.csv'.format(self.injury), names=['case', 'label'], header=None,
                               dtype={'case': str, 'label': np.int64})['case'].tolist()
          self.data_path = 'valid'

    def __len__(self):
        return len(self.case_list)

    def __getitem__(self, index):
        fpath = '{}/{}/{}.npy'.format(self.data_path, self.plane, self.case_list[index])
        stack = np.load(fpath).astype(float)
        for i in range(stack.shape[0]):
          stack[i] = stack[i]/255.
        stack = np.repeat(stack[:, :, :, np.newaxis], 3, axis=3)
        stack = np.expand_dims(stack, axis=0)

        y = self.labels[index]
        y = np.array(y).reshape(1,1)
        return stack, y

In [ ]:
def get_output (injury, data_type):
        if(data_type == 'train'):
          labels = pd.read_csv('train-{}.csv'.format(injury), names=['case', 'label'], header=None,
                              dtype={'case': str, 'label': np.int64})['label'].tolist()
          
          labels = labels[0:1017]
        
        elif(data_type == 'valid'):
          labels = pd.read_csv('train-{}.csv'.format(injury), names=['case', 'label'], header=None,
                              dtype={'case': str, 'label': np.int64})['label'].tolist()
          
          labels = labels[1017:1130]
        
        else:
          labels = pd.read_csv('valid-{}.csv'.format(injury), names=['case', 'label'], header=None,
                              dtype={'case': str, 'label': np.int64})['label'].tolist()
        
        return labels

##Combining model

In [ ]:
def predictor():
  logistic_input = keras.Input(shape=(3,))
  logistic_output = layers.Dense(1, activation='sigmoid')(logistic_input)

  predictor_model = keras.Model(inputs=logistic_input, outputs=logistic_output)
  predictor_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
  return predictor_model

##ResNet Model

In [ ]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # for batch normalization layer, we assume
    # the input data is in channel last format
    bn_axis = 3

    filters1, filters2, filters3 = filters
  
    x = layers.Conv2D(filters1, kernel_size=(1, 1), strides=(1, 1),
                      padding='valid', name=conv_name_base + '2a')(input_tensor)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters2, kernel_size, strides=(1, 1),
                      padding='same', name=conv_name_base + '2b')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters3, kernel_size=(1, 1), strides=(1, 1),
                      padding='valid', name=conv_name_base + '2c')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = layers.Activation('relu')(x)
    return x


def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    bn_axis = 3

    filters1, filters2, filters3 = filters
  
    x = layers.Conv2D(filters1, kernel_size=(1, 1), strides=strides,
                      padding='valid', name=conv_name_base + '2a')(input_tensor)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters2, kernel_size, strides=(1, 1),
                      padding='same', name=conv_name_base + '2b')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters3, kernel_size=(1, 1), strides=(1, 1),
                      padding='valid', name=conv_name_base + '2c')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)
    
    # resize the input
    shortcut = layers.Conv2D(filters3, kernel_size=(1, 1), strides=strides,
                             padding='valid', name=conv_name_base + '1')(input_tensor)
    shortcut = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut) 

    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

In [ ]:
def ResNet():
   
    img_input = keras.Input(shape=(None, 256, 256, 3))
    n_classes = 1;
    
    bn_axis = 3
    x = layers.Cropping3D(cropping=((0,0),(6,6),(6,6)))(img_input)
    x = layers.ZeroPadding2D(padding=(3, 3), name='conv1_pad')(x[0])
    x = layers.Conv2D(64, (7, 7),
                      strides=(2, 2),
                      padding='valid',
                      kernel_initializer='he_normal',
                      name='conv1')(x)
    x = layers.BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = layers.Activation('relu')(x)
    x = layers.ZeroPadding2D(padding=(1, 1), name='pool1_pad')(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')
    # print(x.shape)
    avg = layers.GlobalAveragePooling2D()(x)
    avg = tf.expand_dims(avg,0)
    max_layer = layers.GlobalMaxPooling1D()(avg)
    x = layers.Flatten()(x)
    x = layers.Dense(2048, activation='relu')(max_layer)
    img_output = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs=img_input, outputs=img_output, name='resnet')
    opt = keras.optimizers.Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model



###Abnormal

In [ ]:
# training models
abnormal_sagittal = ResNet()
abnormal_coronal = ResNet()
abnormal_axial = ResNet()

training_abnormal_sagittal_generator = data_generator('abnormal', 'sagittal','train')
validation_abnormal_sagittal_generator = data_generator('abnormal', 'sagittal','valid')
abnormal_sagittal.history = abnormal_sagittal.fit_generator(generator=training_abnormal_sagittal_generator, validation_data=validation_abnormal_sagittal_generator, epochs=20)

abnormal_sagittal.save('resnet_abnormal_sagittal.h5')

training_abnormal_coronal_generator = data_generator('abnormal', 'coronal','train')
validation_abnormal_coronal_generator = data_generator('abnormal', 'coronal','valid')
abnormal_coronal.history = abnormal_coronal.fit_generator(generator=training_abnormal_coronal_generator, validation_data=validation_abnormal_coronal_generator, epochs=20)

abnormal_coronal.save('resnet_abnormal_coronal.h5')

training_abnormal_axial_generator = data_generator('abnormal', 'axial','train')
validation_abnormal_axial_generator = data_generator('abnormal', 'axial','valid')
abnormal_axial.history = abnormal_axial.fit_generator(generator=training_abnormal_axial_generator, validation_data=validation_abnormal_axial_generator, epochs=20)

abnormal_axial.save('resnet_abnormal_axial.h5')

In [ ]:
# loading trained models
abnormal_sagittal = ResNet()
abnormal_coronal = ResNet()
abnormal_axial = ResNet()

abnormal_sagittal.load_weights("resnet_abnormal_sagittal.h5")
abnormal_coronal.load_weights("resnet_abnormal_coronal.h5")
abnormal_axial.load_weights("resnet_abnormal_axial.h5")

In [ ]:
# combining models
training_abnormal_sagittal_generator = data_generator('abnormal', 'sagittal','train')
validation_abnormal_sagittal_generator = data_generator('abnormal', 'sagittal','valid')

training_abnormal_coronal_generator = data_generator('abnormal', 'coronal','train')
validation_abnormal_coronal_generator = data_generator('abnormal', 'coronal','valid')

training_abnormal_axial_generator = data_generator('abnormal', 'axial','train')
validation_abnormal_axial_generator = data_generator('abnormal', 'axial','valid')

training_sagittal_prediction = abnormal_sagittal.predict_generator(training_abnormal_sagittal_generator)
training_coronal_prediction = abnormal_coronal.predict_generator(training_abnormal_coronal_generator)
training_axial_prediction = abnormal_axial.predict_generator(training_abnormal_axial_generator)

validation_sagittal_prediction = abnormal_sagittal.predict_generator(validation_abnormal_sagittal_generator)
validation_coronal_prediction = abnormal_coronal.predict_generator(validation_abnormal_coronal_generator)
validation_axial_prediction = abnormal_axial.predict_generator(validation_abnormal_axial_generator)

training_input = tf.concat([training_sagittal_prediction, training_coronal_prediction, training_axial_prediction],1)
training_output = tf.convert_to_tensor(get_output('abnormal','train'))
validation_input = tf.concat([validation_sagittal_prediction, validation_coronal_prediction, validation_axial_prediction],1)
validation_output = tf.convert_to_tensor(get_output('abnormal','valid'))

abnormal_model = predictor()
abnormal_model.hisory = abnormal_model.fit(training_input, training_output,epochs=30,validation_data=(validation_input,validation_output),batch_size=1)

abnormal_model.save('combine_resnet_abnormal.h5')

In [ ]:
#load combined model
abnormal_model = predictor()
abnormal_model.load_weights('combine_resnet_abnormal.h5')

In [ ]:
# testing the model
test_abnormal_sagittal_generator = data_generator('abnormal', 'sagittal', 'test')
test_abnormal_coronal_generator = data_generator('abnormal', 'coronal', 'test')
test_abnormal_axial_generator = data_generator('abnormal', 'axial', 'test')

test_sagittal_prediction = abnormal_sagittal.predict_generator(test_abnormal_sagittal_generator)
test_coronal_prediction = abnormal_coronal.predict_generator(test_abnormal_coronal_generator)
test_axial_prediction = abnormal_axial.predict_generator(test_abnormal_axial_generator)

test_input = tf.concat([test_sagittal_prediction, test_coronal_prediction, test_axial_prediction],1)
test_output = tf.convert_to_tensor(get_output('abnormal','test'))

abnormal_model.evaluate(test_input,test_output,batch_size=1)

###ACL

In [ ]:
# training models
acl_sagittal = ResNet()
acl_coronal = ResNet()
acl_axial = ResNet()

training_acl_sagittal_generator = data_generator('acl', 'sagittal','train')
validation_acl_sagittal_generator = data_generator('acl', 'sagittal','valid')
acl_sagittal.history = acl_sagittal.fit_generator(generator=training_acl_sagittal_generator, validation_data=validation_acl_sagittal_generator, epochs=20)

acl_sagittal.save('resnet_acl_sagittal.h5')

training_acl_axial_generator = data_generator('acl', 'axial','train')
validation_acl_axial_generator = data_generator('acl', 'axial','valid')
acl_axial.history = acl_axial.fit_generator(generator=training_acl_axial_generator, validation_data=validation_acl_axial_generator, epochs=20)

acl_axial.save('resnet_acl_axial.h5')

training_acl_coronal_generator = data_generator('acl', 'coronal','train')
validation_acl_coronal_generator = data_generator('acl', 'coronal','valid')
acl_coronal.history = acl_coronal.fit_generator(generator=training_acl_coronal_generator, validation_data=validation_acl_coronal_generator, epochs=20)

acl_coronal.save('resnet_acl_coronal.h5')

In [ ]:
# loading trained models
acl_sagittal = ResNet()
acl_coronal = ResNet()
acl_axial = ResNet()

acl_sagittal.load_weights("resnet_acl_sagittal.h5")
acl_coronal.load_weights("resnet_acl_coronal.h5")
acl_axial.load_weights("resnet_acl_axial.h5")

In [ ]:
# combining models
training_acl_sagittal_generator = data_generator('acl', 'sagittal','train')
validation_acl_sagittal_generator = data_generator('acl', 'sagittal','valid')

training_acl_coronal_generator = data_generator('acl', 'coronal','train')
validation_acl_coronal_generator = data_generator('acl', 'coronal','valid')

training_acl_axial_generator = data_generator('acl', 'axial','train')
validation_acl_axial_generator = data_generator('acl', 'axial','valid')

training_sagittal_prediction = acl_sagittal.predict_generator(training_acl_sagittal_generator)
training_coronal_prediction = acl_coronal.predict_generator(training_acl_coronal_generator)
training_axial_prediction = acl_axial.predict_generator(training_acl_axial_generator)

validation_sagittal_prediction = acl_sagittal.predict_generator(validation_acl_sagittal_generator)
validation_coronal_prediction = acl_coronal.predict_generator(validation_acl_coronal_generator)
validation_axial_prediction = acl_axial.predict_generator(validation_acl_axial_generator)

training_input = tf.concat([training_sagittal_prediction, training_coronal_prediction, training_axial_prediction],1)
training_output = tf.convert_to_tensor(get_output('acl','train'))
validation_input = tf.concat([validation_sagittal_prediction, validation_coronal_prediction, validation_axial_prediction],1)
validation_output = tf.convert_to_tensor(get_output('acl','valid'))

acl_model = predictor()
acl_model.hisory = acl_model.fit(training_input, training_output,epochs=30,validation_data=(validation_input,validation_output),batch_size=1)

acl_model.save('combine_resnet_acl.h5')

In [ ]:
#load combined model
acl_model = predictor()
acl_model.load_weights('combine_resnet_acl.h5')

In [ ]:
# testing the model
test_acl_sagittal_generator = data_generator('acl', 'sagittal', 'test')
test_acl_coronal_generator = data_generator('acl', 'coronal', 'test')
test_acl_axial_generator = data_generator('acl', 'axial', 'test')

test_sagittal_prediction = acl_sagittal.predict_generator(test_acl_sagittal_generator)
test_coronal_prediction = acl_coronal.predict_generator(test_acl_coronal_generator)
test_axial_prediction = acl_axial.predict_generator(test_acl_axial_generator)

test_input = tf.concat([test_sagittal_prediction, test_coronal_prediction, test_axial_prediction],1)
test_output = tf.convert_to_tensor(get_output('acl','test'))

acl_model.evaluate(test_input,test_output,batch_size=1)

###Meniscus

In [ ]:
# training models
meniscus_sagittal = ResNet()
meniscus_coronal = ResNet()
meniscus_axial = ResNet()

training_meniscus_sagittal_generator = data_generator('meniscus', 'sagittal','train')
validation_meniscus_sagittal_generator = data_generator('meniscus', 'sagittal','valid')
meniscus_sagittal.history = meniscus_sagittal.fit_generator(generator=training_meniscus_sagittal_generator, validation_data=validation_meniscus_sagittal_generator, epochs=20)

meniscus_sagittal.save('resnet_meniscus_sagittal.h5')

training_meniscus_axial_generator = data_generator('meniscus', 'axial','train')
validation_meniscus_axial_generator = data_generator('meniscus', 'axial','valid')
meniscus_axial.history = meniscus_axial.fit_generator(generator=training_meniscus_axial_generator, validation_data=validation_meniscus_axial_generator, epochs=20)

meniscus_axial.save('resnet_meniscus_axial.h5')

training_meniscus_coronal_generator = data_generator('meniscus', 'coronal','train')
validation_meniscus_coronal_generator = data_generator('meniscus', 'coronal','valid')
meniscus_coronal.history = meniscus_coronal.fit_generator(generator=training_meniscus_coronal_generator, validation_data=validation_meniscus_coronal_generator, epochs=20)

meniscus_coronal.save('resnet_meniscus_coronal.h5')

In [ ]:
# loading trained models
meniscus_sagittal = ResNet()
meniscus_coronal = ResNet()
meniscus_axial = ResNet()

meniscus_sagittal.load_weights("resnet_meniscus_sagittal.h5")
meniscus_coronal.load_weights("resnet_meniscus_coronal.h5")
meniscus_axial.load_weights("resnet_meniscus_axial.h5")

In [ ]:
# combining models
training_meniscus_sagittal_generator = data_generator('meniscus', 'sagittal','train')
validation_meniscus_sagittal_generator = data_generator('meniscus', 'sagittal','valid')

training_meniscus_coronal_generator = data_generator('meniscus', 'coronal','train')
validation_meniscus_coronal_generator = data_generator('meniscus', 'coronal','valid')

training_meniscus_axial_generator = data_generator('meniscus', 'axial','train')
validation_meniscus_axial_generator = data_generator('meniscus', 'axial','valid')

training_sagittal_prediction = meniscus_sagittal.predict_generator(training_meniscus_sagittal_generator)
training_coronal_prediction = meniscus_coronal.predict_generator(training_meniscus_coronal_generator)
training_axial_prediction = meniscus_axial.predict_generator(training_meniscus_axial_generator)

validation_sagittal_prediction = meniscus_sagittal.predict_generator(validation_meniscus_sagittal_generator)
validation_coronal_prediction = meniscus_coronal.predict_generator(validation_meniscus_coronal_generator)
validation_axial_prediction = meniscus_axial.predict_generator(validation_meniscus_axial_generator)

training_input = tf.concat([training_sagittal_prediction, training_coronal_prediction, training_axial_prediction],1)
training_output = tf.convert_to_tensor(get_output('meniscus','train'))
validation_input = tf.concat([validation_sagittal_prediction, validation_coronal_prediction, validation_axial_prediction],1)
validation_output = tf.convert_to_tensor(get_output('meniscus','valid'))

meniscus_model = predictor()
meniscus_model.hisory = meniscus_model.fit(training_input, training_output,epochs=30,validation_data=(validation_input,validation_output),batch_size=1)

meniscus_model.save('combine_resnet_meniscus.h5')

In [ ]:
#load combined model
meniscus_model = predictor()
meniscus_model.load_weights('combine_resnet_meniscus.h5')

In [ ]:
# testing the model
test_meniscus_sagittal_generator = data_generator('meniscus', 'sagittal', 'test')
test_meniscus_coronal_generator = data_generator('meniscus', 'coronal', 'test')
test_meniscus_axial_generator = data_generator('meniscus', 'axial', 'test')

test_sagittal_prediction = meniscus_sagittal.predict_generator(test_meniscus_sagittal_generator)
test_coronal_prediction = meniscus_coronal.predict_generator(test_meniscus_coronal_generator)
test_axial_prediction = meniscus_axial.predict_generator(test_meniscus_axial_generator)

test_input = tf.concat([test_sagittal_prediction, test_coronal_prediction, test_axial_prediction],1)
test_output = tf.convert_to_tensor(get_output('meniscus','test'))

meniscus_model.evaluate(test_input,test_output,batch_size=1)

##VGG Model

In [ ]:
# function for creating a vgg block
def vgg_block(layer_in, n_filters, n_conv):
	# add convolutional layers
	for _ in range(n_conv):
		layer_in = layers.Conv2D(n_filters, (3,3), padding='same', activation='relu')(layer_in)
	# add max pooling layer
	layer_in = layers.MaxPooling2D((2,2), strides=(2,2))(layer_in)
	return layer_in
 
# define model input
def vgg_model():
	visible = keras.Input(shape=(None, 256, 256, 3))
	layer = vgg_block(visible[0], 64, 2)
	layer = vgg_block(layer, 128, 2)
	layer = vgg_block(layer, 256, 3)
	layer = vgg_block(layer, 512, 3)
	layer = vgg_block(layer, 512, 3)

	avg = layers.GlobalAveragePooling2D()(layer)
	avg = tf.expand_dims(avg,0)
	max_layer = layers.GlobalMaxPooling1D()(avg)
	layer = layers.Dense(512, activation='relu')(max_layer)
	out = layers.Dense(1, activation='sigmoid')(layer)

	vgg_model = models.Model(inputs=visible, outputs=out)
	vgg_model.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])
	return vgg_model

###Abnormal

In [ ]:
# training models
abnormal_sagittal = vgg_model()
abnormal_coronal = vgg_model()
abnormal_axial = vgg_model()

training_abnormal_sagittal_generator = data_generator('abnormal', 'sagittal','train')
validation_abnormal_sagittal_generator = data_generator('abnormal', 'sagittal','valid')
abnormal_sagittal.history = abnormal_sagittal.fit_generator(generator=training_abnormal_sagittal_generator, validation_data=validation_abnormal_sagittal_generator, epochs=5)

abnormal_sagittal.save('vgg_abnormal_sagittal.h5')

training_abnormal_coronal_generator = data_generator('abnormal', 'coronal','train')
validation_abnormal_coronal_generator = data_generator('abnormal', 'coronal','valid')
abnormal_coronal.history = abnormal_coronal.fit_generator(generator=training_abnormal_coronal_generator, validation_data=validation_abnormal_coronal_generator, epochs=5)

abnormal_coronal.save('vgg_abnormal_coronal.h5')

training_abnormal_axial_generator = data_generator('abnormal', 'axial','train')
validation_abnormal_axial_generator = data_generator('abnormal', 'axial','valid')
abnormal_axial.history = abnormal_axial.fit_generator(generator=training_abnormal_axial_generator, validation_data=validation_abnormal_axial_generator, epochs=5)

abnormal_axial.save('vgg_abnormal_axial.h5')

In [ ]:
# loading trained models
abnormal_sagittal = vgg_model()
abnormal_coronal = vgg_model()
abnormal_axial = vgg_model()

abnormal_sagittal.load_weights("vgg_abnormal_sagittal.h5")
abnormal_coronal.load_weights("vgg_abnormal_coronal.h5")
abnormal_axial.load_weights("vgg_abnormal_axial.h5")

In [ ]:
# combining models
training_abnormal_sagittal_generator = data_generator('abnormal', 'sagittal','train')
validation_abnormal_sagittal_generator = data_generator('abnormal', 'sagittal','valid')

training_abnormal_coronal_generator = data_generator('abnormal', 'coronal','train')
validation_abnormal_coronal_generator = data_generator('abnormal', 'coronal','valid')

training_abnormal_axial_generator = data_generator('abnormal', 'axial','train')
validation_abnormal_axial_generator = data_generator('abnormal', 'axial','valid')

training_sagittal_prediction = abnormal_sagittal.predict_generator(training_abnormal_sagittal_generator)
training_coronal_prediction = abnormal_coronal.predict_generator(training_abnormal_coronal_generator)
training_axial_prediction = abnormal_axial.predict_generator(training_abnormal_axial_generator)

validation_sagittal_prediction = abnormal_sagittal.predict_generator(validation_abnormal_sagittal_generator)
validation_coronal_prediction = abnormal_coronal.predict_generator(validation_abnormal_coronal_generator)
validation_axial_prediction = abnormal_axial.predict_generator(validation_abnormal_axial_generator)

training_input = tf.concat([training_sagittal_prediction, training_coronal_prediction, training_axial_prediction],1)
training_output = tf.convert_to_tensor(get_output('abnormal','train'))
validation_input = tf.concat([validation_sagittal_prediction, validation_coronal_prediction, validation_axial_prediction],1)
validation_output = tf.convert_to_tensor(get_output('abnormal','valid'))

abnormal_model = predictor()
abnormal_model.hisory = abnormal_model.fit(training_input, training_output,epochs=30,validation_data=(validation_input,validation_output),batch_size=1)

abnormal_model.save('combine_vgg_abnormal.h5')

In [ ]:
#load combined model
abnormal_model = predictor()
abnormal_model.load_weights('combine_vgg_abnormal.h5')

In [ ]:
# testing the model
test_abnormal_sagittal_generator = data_generator('abnormal', 'sagittal', 'test')
test_abnormal_coronal_generator = data_generator('abnormal', 'coronal', 'test')
test_abnormal_axial_generator = data_generator('abnormal', 'axial', 'test')

test_sagittal_prediction = abnormal_sagittal.predict_generator(test_abnormal_sagittal_generator)
test_coronal_prediction = abnormal_coronal.predict_generator(test_abnormal_coronal_generator)
test_axial_prediction = abnormal_axial.predict_generator(test_abnormal_axial_generator)

test_input = tf.concat([test_sagittal_prediction, test_coronal_prediction, test_axial_prediction],1)
test_output = tf.convert_to_tensor(get_output('abnormal','test'))

abnormal_model.evaluate(test_input,test_output,batch_size=1)

###ACL

In [ ]:
# training models
acl_sagittal = vgg_model()
acl_coronal = vgg_model()
acl_axial = vgg_model()

training_acl_sagittal_generator = data_generator('acl', 'sagittal','train')
validation_acl_sagittal_generator = data_generator('acl', 'sagittal','valid')
acl_sagittal.history = acl_sagittal.fit_generator(generator=training_acl_sagittal_generator, validation_data=validation_acl_sagittal_generator, epochs=5)

acl_sagittal.save('vgg_acl_sagittal.h5')

training_acl_axial_generator = data_generator('acl', 'axial','train')
validation_acl_axial_generator = data_generator('acl', 'axial','valid')
acl_axial.history = acl_axial.fit_generator(generator=training_acl_axial_generator, validation_data=validation_acl_axial_generator, epochs=5)

acl_axial.save('vgg_acl_axial.h5')

training_acl_coronal_generator = data_generator('acl', 'coronal','train')
validation_acl_coronal_generator = data_generator('acl', 'coronal','valid')
acl_coronal.history = acl_coronal.fit_generator(generator=training_acl_coronal_generator, validation_data=validation_acl_coronal_generator, epochs=5)

acl_coronal.save('vgg_acl_coronal.h5')

In [ ]:
acl_sagittal = vgg_model()
acl_coronal = vgg_model()
acl_axial = vgg_model()

acl_sagittal.load_weights("vgg_acl_sagittal.h5")
acl_coronal.load_weights("vgg_acl_coronal.h5")
acl_axial.load_weights("vgg_acl_axial.h5")

In [ ]:
# combining models
training_acl_sagittal_generator = data_generator('acl', 'sagittal','train')
validation_acl_sagittal_generator = data_generator('acl', 'sagittal','valid')

training_acl_coronal_generator = data_generator('acl', 'coronal','train')
validation_acl_coronal_generator = data_generator('acl', 'coronal','valid')

training_acl_axial_generator = data_generator('acl', 'axial','train')
validation_acl_axial_generator = data_generator('acl', 'axial','valid')

training_sagittal_prediction = acl_sagittal.predict_generator(training_acl_sagittal_generator)
training_coronal_prediction = acl_coronal.predict_generator(training_acl_coronal_generator)
training_axial_prediction = acl_axial.predict_generator(training_acl_axial_generator)

validation_sagittal_prediction = acl_sagittal.predict_generator(validation_acl_sagittal_generator)
validation_coronal_prediction = acl_coronal.predict_generator(validation_acl_coronal_generator)
validation_axial_prediction = acl_axial.predict_generator(validation_acl_axial_generator)

training_input = tf.concat([training_sagittal_prediction, training_coronal_prediction, training_axial_prediction],1)
training_output = tf.convert_to_tensor(get_output('acl','train'))
validation_input = tf.concat([validation_sagittal_prediction, validation_coronal_prediction, validation_axial_prediction],1)
validation_output = tf.convert_to_tensor(get_output('acl','valid'))

acl_model = predictor()
acl_model.hisory = acl_model.fit(training_input, training_output,epochs=30,validation_data=(validation_input,validation_output),batch_size=1)

acl_model.save('combine_vgg_acl.h5')

In [ ]:
#load combined model
acl_model = predictor()
acl_model.load_weights('combine_vgg_acl.h5')

In [ ]:
# testing the model
test_acl_sagittal_generator = data_generator('acl', 'sagittal', 'test')
test_acl_coronal_generator = data_generator('acl', 'coronal', 'test')
test_acl_axial_generator = data_generator('acl', 'axial', 'test')

test_sagittal_prediction = acl_sagittal.predict_generator(test_acl_sagittal_generator)
test_coronal_prediction = acl_coronal.predict_generator(test_acl_coronal_generator)
test_axial_prediction = acl_axial.predict_generator(test_acl_axial_generator)

test_input = tf.concat([test_sagittal_prediction, test_coronal_prediction, test_axial_prediction],1)
test_output = tf.convert_to_tensor(get_output('acl','test'))

acl_model.evaluate(test_input,test_output,batch_size=1)

120/120 [==============================] - 0s 2ms/step - loss: 0.8610 - accuracy: 0.5500


[0.861045241355896, 0.550000011920929]

###Meniscus

In [ ]:
# training models
meniscus_sagittal = vgg_model()
meniscus_coronal = vgg_model()
meniscus_axial = vgg_model()

training_meniscus_sagittal_generator = data_generator('meniscus', 'sagittal','train')
validation_meniscus_sagittal_generator = data_generator('meniscus', 'sagittal','valid')
meniscus_axial.history = meniscus_sagittal.fit_generator(generator=training_meniscus_sagittal_generator, validation_data=validation_meniscus_sagittal_generator, epochs=5)

meniscus_sagittal.save('vgg_meniscus_sagittal.h5')

training_meniscus_axial_generator = data_generator('meniscus', 'axial','train')
validation_meniscus_axial_generator = data_generator('meniscus', 'axial','valid')
meniscus_axial.history = meniscus_axial.fit_generator(generator=training_meniscus_axial_generator, validation_data=validation_meniscus_axial_generator, epochs=5)

meniscus_axial.save('vgg_meniscus_axial.h5')

training_meniscus_coronal_generator = data_generator('meniscus', 'coronal','train')
validation_meniscus_coronal_generator = data_generator('meniscus', 'coronal','valid')
meniscus_coronal.history = meniscus_coronal.fit_generator(generator=training_meniscus_coronal_generator, validation_data=validation_meniscus_coronal_generator, epochs=5)

meniscus_coronal.save('vgg_meniscus_coronal.h5')

In [ ]:
# loading trained models
meniscus_sagittal = vgg_model()
meniscus_coronal = vgg_model()
meniscus_axial = vgg_model()

meniscus_sagittal.load_weights("vgg_meniscus_sagittal.h5")
meniscus_coronal.load_weights("vgg_meniscus_coronal.h5")
meniscus_axial.load_weights("vgg_meniscus_axial.h5")

In [ ]:
# combining models
training_meniscus_sagittal_generator = data_generator('meniscus', 'sagittal','train')
validation_meniscus_sagittal_generator = data_generator('meniscus', 'sagittal','valid')

training_meniscus_coronal_generator = data_generator('meniscus', 'coronal','train')
validation_meniscus_coronal_generator = data_generator('meniscus', 'coronal','valid')

training_meniscus_axial_generator = data_generator('meniscus', 'axial','train')
validation_meniscus_axial_generator = data_generator('meniscus', 'axial','valid')

training_sagittal_prediction = meniscus_sagittal.predict_generator(training_meniscus_sagittal_generator)
training_coronal_prediction = meniscus_coronal.predict_generator(training_meniscus_coronal_generator)
training_axial_prediction = meniscus_axial.predict_generator(training_meniscus_axial_generator)

validation_sagittal_prediction = meniscus_sagittal.predict_generator(validation_meniscus_sagittal_generator)
validation_coronal_prediction = meniscus_coronal.predict_generator(validation_meniscus_coronal_generator)
validation_axial_prediction = meniscus_axial.predict_generator(validation_meniscus_axial_generator)

training_input = tf.concat([training_sagittal_prediction, training_coronal_prediction, training_axial_prediction],1)
training_output = tf.convert_to_tensor(get_output('meniscus','train'))
validation_input = tf.concat([validation_sagittal_prediction, validation_coronal_prediction, validation_axial_prediction],1)
validation_output = tf.convert_to_tensor(get_output('meniscus','valid'))

meniscus_model = predictor()
meniscus_model.hisory = meniscus_model.fit(training_input, training_output,epochs=100,validation_data=(validation_input,validation_output),batch_size=1)

meniscus_model.save('combine_vgg_meniscus.h5')

In [ ]:
#load combined model
meniscus_model = predictor()
meniscus_model.load_weights('combine_vgg_meniscus.h5')

In [ ]:
# testing the model
test_meniscus_sagittal_generator = data_generator('meniscus', 'sagittal', 'test')
test_meniscus_coronal_generator = data_generator('meniscus', 'coronal', 'test')
test_meniscus_axial_generator = data_generator('meniscus', 'axial', 'test')

test_sagittal_prediction = meniscus_sagittal.predict_generator(test_meniscus_sagittal_generator)
test_coronal_prediction = meniscus_coronal.predict_generator(test_meniscus_coronal_generator)
test_axial_prediction = meniscus_axial.predict_generator(test_meniscus_axial_generator)

test_input = tf.concat([test_sagittal_prediction, test_coronal_prediction, test_axial_prediction],1)
test_output = tf.convert_to_tensor(get_output('meniscus','test'))

meniscus_model.evaluate(test_input,test_output,batch_size=1)

120/120 [==============================] - 0s 2ms/step - loss: 0.6980 - accuracy: 0.5667


[0.6979799270629883, 0.5666666626930237]

##Inception Model

In [ ]:
def BNConv(nb_filter, nb_row, nb_col, strides=(1, 1, 1), padding="valid"):
    def f(input):
        conv = layers.Conv3D(filters=nb_filter, kernel_size=(1,nb_row,nb_col), strides=strides,
                      padding=padding)(input)
        x = layers.BatchNormalization(axis=2)(conv)
        return layers.Activation('relu')(x) #modified apply batchNormalization then activation
    return f

def InceptionFig5(filters):
    def f(input):

        # Tower A
        conv_a1 = BNConv(64, 1, 1, padding="same")(input)
        conv_a2 = BNConv(96, 3, 3, padding="same")(conv_a1)
        conv_a3 = BNConv(96, 3, 3, padding="same")(conv_a2)

        # Tower B
        conv_b1 = BNConv(48, 1, 1, padding="same")(input)
        conv_b2 = BNConv(64, 3, 3, padding="same")(conv_b1)

        # Tower C
        pool_c1 = layers.AveragePooling3D(pool_size=(1, 3, 3), strides=(1, 1, 1), padding="same")(input)
        conv_c2 = BNConv(filters, 1, 1, padding="same")(pool_c1)#modified

        # Tower D
        conv_d1 = BNConv(64, 1, 1, padding="same")(input)

        return keras.layers.concatenate([conv_a3, conv_b2, conv_c2, conv_d1], axis=4)

    return f

def DimReductionA():
    def f(input):
        conv_a1 = BNConv(64, 1, 1, padding="same")(input)
        conv_a2 = BNConv(96, 3, 3, padding="same")(conv_a1)
        conv_a3 = BNConv(96, 3, 3, strides=(1, 2, 2), padding="valid")(conv_a2)

        # another inconsistency between (model.txt + mxnet diagram) and (the paper)
        # the Fig 10 in the paper shows a 1x1 convolution before
        # the 3x3. Going with model.txt + mxnet diagram
        conv_b = BNConv(384, 3, 3, strides=(1, 2, 2), padding ="valid")(input)

        pool_c = layers.MaxPooling3D(pool_size=(1, 3, 3), strides=(1, 2, 2), padding="valid")(input)

        return keras.layers.concatenate([conv_a3, conv_b, pool_c], axis=4)
    return f

def InceptionFig6(filters):
    def f(input):
        conv_a1 = BNConv(filters, 1, 1, padding="same")(input)
        conv_a2 = BNConv(filters, 1, 7, padding="same")(conv_a1)
        conv_a3 = BNConv(filters, 7, 1, padding="same")(conv_a2)
        conv_a4 = BNConv(filters, 1, 7, padding="same")(conv_a3)
        conv_a5 = BNConv(192, 7, 1, padding="same")(conv_a4)

        # Tower B
        conv_b1 = BNConv(filters, 1, 1, padding="same")(input)
        conv_b2 = BNConv(filters, 1, 7, padding="same")(conv_b1)
        conv_b3 = BNConv(192, 7, 1, padding="same")(conv_b2)

        # Tower C
        pool_c1 = layers.AveragePooling3D(pool_size=(1, 3, 3), strides=(1, 1, 1), padding="same")(input)
        conv_c2 = BNConv(192, 1, 1)(pool_c1)

        # Tower D
        conv_d = BNConv(192, 1, 1)(input)

        return keras.layers.concatenate([conv_a5, conv_b3, conv_c2, conv_d], axis=4)

    return f

def DimReductionB():
    def f(input):
        # Tower A
        conv_a1 = BNConv(192, 1, 1)(input)
        conv_a2 = BNConv(320, 3, 3, strides=(1, 2, 2), padding="valid")(conv_a1)

        # Tower B
        conv_b1 = BNConv(192, 1, 1, padding="same")(input)
        conv_b2 = BNConv(192, 1, 7, padding="same")(conv_b1)
        conv_b3 = BNConv(192, 7, 1, padding="same")(conv_b2)
        conv_b4 = BNConv(192, 3, 3, strides=(1, 2, 2), padding="valid")(conv_b3)

        # Tower C
        pool_c = layers.MaxPooling3D(pool_size=(1, 3, 3), strides=(1, 2, 2), padding="valid")(input)

        return keras.layers.concatenate([conv_a2, conv_b4, pool_c], axis=4)
    return f

def InceptionFig7():
    def f(input):
        # Tower A
        conv_a1 = BNConv(448, 1, 1, padding="same")(input)
        conv_a2 = BNConv(384, 3, 3, padding="same")(conv_a1)
        conv_a3 = BNConv(384, 1, 3, padding="same")(conv_a2)
        conv_a4 = BNConv(384, 3, 1, padding="same")(conv_a2)#modified conv_a2 instead of conv_a3

        # Tower B
        conv_b1 = BNConv(384, 1, 1, padding="same")(input)
        conv_b2 = BNConv(384, 1, 3, padding="same")(conv_b1)
        conv_b3 = BNConv(384, 3, 1, padding="same")(conv_b1)#modified conv_b1 instead of conv_b2

        # Tower C
        pool_c1 = layers.AveragePooling3D(pool_size=(1, 3, 3), strides=(1, 1, 1), padding="same")(input)
        conv_c2 = BNConv(192, 1, 1)(pool_c1)

        # Tower D
        conv_d = BNConv(320, 1, 1)(input)

        return keras.layers.concatenate([conv_a4, conv_a3, conv_b3, conv_b2, conv_c2, conv_d], axis=4)#modified

    return f

Using TensorFlow backend.


In [ ]:
def inception_v3():    
    input = keras.Input(shape=(None,256, 256, 3))
    conv_1 = BNConv(32, 3, 3, strides=(1, 2, 2))(input)
    conv_2 = BNConv(32, 3, 3)(conv_1)
    conv_3 = BNConv(64, 3, 3, padding="same")(conv_2)
    pool_4 = layers.MaxPooling3D(pool_size=(1, 3, 3), strides=(1, 2, 2), padding="valid")(conv_3)
    conv_5 = BNConv(80, 1, 1)(pool_4)
    conv_6 = BNConv(192, 3, 3)(conv_5)
    pool_7 = layers.MaxPooling3D(pool_size=(1,3, 3), strides=(1,2, 2), padding="valid")(conv_6)
    inception_8 = InceptionFig5(filters = 32)(pool_7)
    inception_9 = InceptionFig5(filters = 64)(inception_8)
    inception_10 = InceptionFig5(filters = 64)(inception_9)
    inception_11 = DimReductionA()(inception_10)
    inception_12 = InceptionFig6(128)(inception_11)
    inception_13 = InceptionFig6(160)(inception_12)
    inception_14 = InceptionFig6(160)(inception_13)
    inception_15 = InceptionFig6(192)(inception_14)
    inception_17 = DimReductionB()(inception_15)
    inception_18 = InceptionFig7()(inception_17)
    inception_19 = InceptionFig7()(inception_18)
    #**** modified *****
    x = layers.AveragePooling3D(strides=(1,6,6))(inception_19)
    x = layers.GlobalMaxPooling3D()(x)
    pooling_output = layers.Dense(1, activation='sigmoid')(x)
    model = keras.Model(inputs= input, outputs=pooling_output, name = 'inception_v3')
    model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=["accuracy"])
    return model

###Abnormal

In [ ]:
# training models
abnormal_sagittal = inception_v3()
abnormal_coronal = inception_v3()
abnormal_axial = inception_v3()

training_abnormal_sagittal_generator = data_generator('abnormal', 'sagittal','train')
validation_abnormal_sagittal_generator = data_generator('abnormal', 'sagittal','valid')
abnormal_sagittal.history = abnormal_sagittal.fit_generator(generator=training_abnormal_sagittal_generator, validation_data=validation_abnormal_sagittal_generator, epochs=1)

abnormal_sagittal.save('inception_abnormal_sagittal.h5')

training_abnormal_coronal_generator = data_generator('abnormal', 'coronal','train')
validation_abnormal_coronal_generator = data_generator('abnormal', 'coronal','valid')
abnormal_coronal.history = abnormal_coronal.fit_generator(generator=training_abnormal_coronal_generator, validation_data=validation_abnormal_coronal_generator, epochs=1)

abnormal_coronal.save('inception_abnormal_coronal.h5')

training_abnormal_axial_generator = data_generator('abnormal', 'axial','train')
validation_abnormal_axial_generator = data_generator('abnormal', 'axial','valid')
abnormal_axial.history = abnormal_axial.fit_generator(generator=training_abnormal_axial_generator, validation_data=validation_abnormal_axial_generator, epochs=1)

abnormal_axial.save('inception_abnormal_axial.h5')

In [ ]:
# loading trained models
abnormal_sagittal = inception_v3()
abnormal_coronal = inception_v3()
abnormal_axial = inception_v3()

abnormal_sagittal.load_weights("inception_abnormal_sagittal.h5")
abnormal_coronal.load_weights("inception_abnormal_coronal.h5")
abnormal_axial.load_weights("inception_abnormal_axial.h5")

In [ ]:
# combining models
training_abnormal_sagittal_generator = data_generator('abnormal', 'sagittal','train')
validation_abnormal_sagittal_generator = data_generator('abnormal', 'sagittal','valid')

training_abnormal_coronal_generator = data_generator('abnormal', 'coronal','train')
validation_abnormal_coronal_generator = data_generator('abnormal', 'coronal','valid')

training_abnormal_axial_generator = data_generator('abnormal', 'axial','train')
validation_abnormal_axial_generator = data_generator('abnormal', 'axial','valid')

training_sagittal_prediction = abnormal_sagittal.predict_generator(training_abnormal_sagittal_generator)
training_coronal_prediction = abnormal_coronal.predict_generator(training_abnormal_coronal_generator)
training_axial_prediction = abnormal_axial.predict_generator(training_abnormal_axial_generator)

validation_sagittal_prediction = abnormal_sagittal.predict_generator(validation_abnormal_sagittal_generator)
validation_coronal_prediction = abnormal_coronal.predict_generator(validation_abnormal_coronal_generator)
validation_axial_prediction = abnormal_axial.predict_generator(validation_abnormal_axial_generator)

training_input = tf.concat([training_sagittal_prediction, training_coronal_prediction, training_axial_prediction],1)
training_output = tf.convert_to_tensor(get_output('abnormal','train'))
validation_input = tf.concat([validation_sagittal_prediction, validation_coronal_prediction, validation_axial_prediction],1)
validation_output = tf.convert_to_tensor(get_output('abnormal','valid'))

abnormal_model = predictor()
abnormal_model.hisory = abnormal_model.fit(training_input, training_output,epochs=30,validation_data=(validation_input,validation_output),batch_size=1)

abnormal_model.save('combine_inception_abnormal.h5')

In [ ]:
#load combined model
abnormal_model = predictor()
abnormal_model.load_weights('combine_inception_abnormal.h5')

In [ ]:
# testing the model
test_abnormal_sagittal_generator = data_generator('abnormal', 'sagittal', 'test')
test_abnormal_coronal_generator = data_generator('abnormal', 'coronal', 'test')
test_abnormal_axial_generator = data_generator('abnormal', 'axial', 'test')

test_sagittal_prediction = abnormal_sagittal.predict_generator(test_abnormal_sagittal_generator)
test_coronal_prediction = abnormal_coronal.predict_generator(test_abnormal_coronal_generator)
test_axial_prediction = abnormal_axial.predict_generator(test_abnormal_axial_generator)

test_input = tf.concat([test_sagittal_prediction, test_coronal_prediction, test_axial_prediction],1)
test_output = tf.convert_to_tensor(get_output('abnormal','test'))

abnormal_model.evaluate(test_input,test_output,batch_size=1)

###ACL

In [ ]:
# training models
acl_sagittal = inception_v3()
acl_coronal = inception_v3()
acl_axial = inception_v3()

training_acl_sagittal_generator = data_generator('acl', 'sagittal','train')
validation_acl_sagittal_generator = data_generator('acl', 'sagittal','valid')
acl_sagittal.history = acl_sagittal.fit_generator(generator=training_acl_sagittal_generator, validation_data=validation_acl_sagittal_generator, epochs=1)

acl_sagittal.save('inception_acl_sagittal.h5')

training_acl_axial_generator = data_generator('acl', 'axial','train')
validation_acl_axial_generator = data_generator('acl', 'axial','valid')
acl_axial.history = acl_axial.fit_generator(generator=training_acl_axial_generator, validation_data=validation_acl_axial_generator, epochs=1)

acl_axial.save('inception_acl_axial.h5')

training_acl_coronal_generator = data_generator('acl', 'coronal','train')
validation_acl_coronal_generator = data_generator('acl', 'coronal','valid')
acl_coronal.history = acl_coronal.fit_generator(generator=training_acl_coronal_generator, validation_data=validation_acl_coronal_generator, epochs=1)

acl_coronal.save('inception_acl_coronal.h5')

In [ ]:
# loading trained models
acl_sagittal = inception_v3()
acl_coronal = inception_v3()
acl_axial = inception_v3()

acl_sagittal.load_weights("inception_acl_sagittal.h5")
acl_coronal.load_weights("inception_acl_coronal.h5")
acl_axial.load_weights("inception_acl_axial.h5")

In [ ]:
# combining models
training_acl_sagittal_generator = data_generator('acl', 'sagittal','train')
validation_acl_sagittal_generator = data_generator('acl', 'sagittal','valid')

training_acl_coronal_generator = data_generator('acl', 'coronal','train')
validation_acl_coronal_generator = data_generator('acl', 'coronal','valid')

training_acl_axial_generator = data_generator('acl', 'axial','train')
validation_acl_axial_generator = data_generator('acl', 'axial','valid')

training_sagittal_prediction = acl_sagittal.predict_generator(training_acl_sagittal_generator)
training_coronal_prediction = acl_coronal.predict_generator(training_acl_coronal_generator)
training_axial_prediction = acl_axial.predict_generator(training_acl_axial_generator)

validation_sagittal_prediction = acl_sagittal.predict_generator(validation_acl_sagittal_generator)
validation_coronal_prediction = acl_coronal.predict_generator(validation_acl_coronal_generator)
validation_axial_prediction = acl_axial.predict_generator(validation_acl_axial_generator)

training_input = tf.concat([training_sagittal_prediction, training_coronal_prediction, training_axial_prediction],1)
training_output = tf.convert_to_tensor(get_output('acl','train'))
validation_input = tf.concat([validation_sagittal_prediction, validation_coronal_prediction, validation_axial_prediction],1)
validation_output = tf.convert_to_tensor(get_output('acl','valid'))

acl_model = predictor()
acl_model.hisory = acl_model.fit(training_input, training_output,epochs=30,validation_data=(validation_input,validation_output),batch_size=1)

meniscus_model.save('combine_inception_acl.h5')

In [ ]:
#load combined model
acl_model = predictor()
acl_model.load_weights('combine_inception_acl.h5')

In [ ]:
# testing the model
test_acl_sagittal_generator = data_generator('acl', 'sagittal', 'test')
test_acl_coronal_generator = data_generator('acl', 'coronal', 'test')
test_acl_axial_generator = data_generator('acl', 'axial', 'test')

test_sagittal_prediction = acl_sagittal.predict_generator(test_acl_sagittal_generator)
test_coronal_prediction = acl_coronal.predict_generator(test_acl_coronal_generator)
test_axial_prediction = acl_axial.predict_generator(test_acl_axial_generator)

test_input = tf.concat([test_sagittal_prediction, test_coronal_prediction, test_axial_prediction],1)
test_output = tf.convert_to_tensor(get_output('acl','test'))

acl_model.evaluate(test_input,test_output,batch_size=1)

###Meniscus

In [ ]:
# training models
meniscus_sagittal = inception_v3()
meniscus_coronal = inception_v3()
meniscus_axial = inception_v3()

training_meniscus_sagittal_generator = data_generator('meniscus', 'sagittal','train')
validation_meniscus_sagittal_generator = data_generator('meniscus', 'sagittal','valid')
meniscus_sagittal.history = meniscus_sagittal.fit_generator(generator=training_meniscus_sagittal_generator, validation_data=validation_meniscus_sagittal_generator, epochs=1)

meniscus_sagittal.save('inception_meniscus_sagittal.h5')

training_meniscus_axial_generator = data_generator('meniscus', 'axial','train')
validation_meniscus_axial_generator = data_generator('meniscus', 'axial','valid')
meniscus_axial.history = meniscus_axial.fit_generator(generator=training_meniscus_axial_generator, validation_data=validation_meniscus_axial_generator, epochs=1)

meniscus_axial.save('inception_meniscus_axial.h5')

training_meniscus_coronal_generator = data_generator('meniscus', 'coronal','train')
validation_meniscus_coronal_generator = data_generator('meniscus', 'coronal','valid')
meniscus_coronal.history = meniscus_coronal.fit_generator(generator=training_meniscus_coronal_generator, validation_data=validation_meniscus_coronal_generator, epochs=1)

meniscus_coronal.save('inception_meniscus_coronal.h5')

In [ ]:
# loading trained models
meniscus_sagittal = inception_v3()
meniscus_coronal = inception_v3()
meniscus_axial = inception_v3()

meniscus_sagittal.load_weights("inception_meniscus_sagittal.h5")
meniscus_coronal.load_weights("inception_meniscus_coronal.h5")
meniscus_axial.load_weights("inception_meniscus_axial.h5")

In [ ]:
# combining models
training_meniscus_sagittal_generator = data_generator('meniscus', 'sagittal','train')
validation_meniscus_sagittal_generator = data_generator('meniscus', 'sagittal','valid')

training_meniscus_coronal_generator = data_generator('meniscus', 'coronal','train')
validation_meniscus_coronal_generator = data_generator('meniscus', 'coronal','valid')

training_meniscus_axial_generator = data_generator('meniscus', 'axial','train')
validation_meniscus_axial_generator = data_generator('meniscus', 'axial','valid')

training_sagittal_prediction = meniscus_sagittal.predict_generator(training_meniscus_sagittal_generator)
training_coronal_prediction = meniscus_coronal.predict_generator(training_meniscus_coronal_generator)
training_axial_prediction = meniscus_axial.predict_generator(training_meniscus_axial_generator)

validation_sagittal_prediction = meniscus_sagittal.predict_generator(validation_meniscus_sagittal_generator)
validation_coronal_prediction = meniscus_coronal.predict_generator(validation_meniscus_coronal_generator)
validation_axial_prediction = meniscus_axial.predict_generator(validation_meniscus_axial_generator)

training_input = tf.concat([training_sagittal_prediction, training_coronal_prediction, training_axial_prediction],1)
training_output = tf.convert_to_tensor(get_output('meniscus','train'))
validation_input = tf.concat([validation_sagittal_prediction, validation_coronal_prediction, validation_axial_prediction],1)
validation_output = tf.convert_to_tensor(get_output('meniscus','valid'))

meniscus_model = predictor()
meniscus_model.hisory = meniscus_model.fit(training_input, training_output,epochs=30,validation_data=(validation_input,validation_output),batch_size=1)

meniscus_model.save('combine_inception_meniscus.h5')

In [ ]:
#load combined model
meniscus_model = predictor()
meniscus_model.load_weights('combine_inception_meniscus.h5')

In [ ]:
# testing the model
test_meniscus_sagittal_generator = data_generator('meniscus', 'sagittal', 'test')
test_meniscus_coronal_generator = data_generator('meniscus', 'coronal', 'test')
test_meniscus_axial_generator = data_generator('meniscus', 'axial', 'test')

test_sagittal_prediction = meniscus_sagittal.predict_generator(test_meniscus_sagittal_generator)
test_coronal_prediction = meniscus_coronal.predict_generator(test_meniscus_coronal_generator)
test_axial_prediction = meniscus_axial.predict_generator(test_meniscus_axial_generator)

test_input = tf.concat([test_sagittal_prediction, test_coronal_prediction, test_axial_prediction],1)
test_output = tf.convert_to_tensor(get_output('meniscus','test'))

meniscus_model.evaluate(test_input,test_output,batch_size=1)

## AlexNet

In [ ]:
def mrnet_model():
  alexnet_input = keras.Input(shape=(None,256, 256, 3))
  x = layers.Cropping3D(cropping=((0,0),(15,14),(15,14)))(alexnet_input)
  x = layers.Conv3D(filters=96, kernel_size=(1,11,11), strides=(1,4,4), padding='valid', activation='relu')(x)
  x = layers.MaxPooling3D(pool_size=(1,3,3), strides=(1,2,2), padding='valid')(x)
  x = layers.Conv3D(filters=256, kernel_size=(1,5,5), padding='valid', activation='relu')(x)
  x = layers.MaxPooling3D(pool_size=(1,3,3), strides=(1,2,2), padding='valid')(x)
  x = layers.Conv3D(filters=384, kernel_size=(1,3,3), padding='valid', activation='relu')(x)
  alexnet_output = layers.Conv3D(filters=256, kernel_size=(1,3,3), padding='valid', activation='relu')(x)
  alexnet = keras.Model(inputs=alexnet_input, outputs=alexnet_output, name='alexnet')
  x = layers.AveragePooling3D(strides=(1,7,7))(alexnet_output)
  x = layers.GlobalMaxPooling3D()(x)
  pooling_output = layers.Dense(1, activation='sigmoid')(x)
  mrnet = keras.Model(inputs=alexnet_input, outputs=pooling_output)
  mrnet.compile(optimizer='sgd', loss='binary_crossentropy', metrics=["accuracy"])
  return mrnet

def predictor():
  logistic_input = keras.Input(shape=(3,))
  logistic_output = layers.Dense(1, activation='sigmoid')(logistic_input)

  predictor_model = keras.Model(inputs=logistic_input, outputs=logistic_output)
  predictor_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
  return predictor_model


###Abnormal

In [ ]:
# training models
abnormal_sagittal = mrnet_model()
abnormal_coronal = mrnet_model()
abnormal_axial = mrnet_model()

training_abnormal_sagittal_generator = data_generator('abnormal', 'sagittal','train')
validation_abnormal_sagittal_generator = data_generator('abnormal', 'sagittal','valid')
abnormal_sagittal.history = abnormal_sagittal.fit_generator(generator=training_abnormal_sagittal_generator, validation_data=validation_abnormal_sagittal_generator, epochs=30)

abnormal_sagittal.save('mrnet_abnormal_sagittal.h5')

training_abnormal_coronal_generator = data_generator('abnormal', 'coronal','train')
validation_abnormal_coronal_generator = data_generator('abnormal', 'coronal','valid')
abnormal_coronal.history = abnormal_coronal.fit_generator(generator=training_abnormal_coronal_generator, validation_data=validation_abnormal_coronal_generator, epochs=30)

abnormal_coronal.save('mrnet_abnormal_coronal.h5')

training_abnormal_axial_generator = data_generator('abnormal', 'axial','train')
validation_abnormal_axial_generator = data_generator('abnormal', 'axial','valid')
abnormal_axial.history = abnormal_axial.fit_generator(generator=training_abnormal_axial_generator, validation_data=validation_abnormal_axial_generator, epochs=30)

abnormal_axial.save('mrnet_abnormal_axial.h5')

In [ ]:
# loading trained models
abnormal_sagittal = mrnet_model()
abnormal_coronal = mrnet_model()
abnormal_axial = mrnet_model()

abnormal_sagittal.load_weights("mrnet_abnormal_sagittal.h5")
abnormal_coronal.load_weights("mrnet_abnormal_coronal.h5")
abnormal_axial.load_weights("mrnet_abnormal_axial.h5")

In [ ]:
# combining models
training_abnormal_sagittal_generator = data_generator('abnormal', 'sagittal','train')
validation_abnormal_sagittal_generator = data_generator('abnormal', 'sagittal','valid')

training_abnormal_coronal_generator = data_generator('abnormal', 'coronal','train')
validation_abnormal_coronal_generator = data_generator('abnormal', 'coronal','valid')

training_abnormal_axial_generator = data_generator('abnormal', 'axial','train')
validation_abnormal_axial_generator = data_generator('abnormal', 'axial','valid')

training_sagittal_prediction = abnormal_sagittal.predict_generator(training_abnormal_sagittal_generator)
training_coronal_prediction = abnormal_coronal.predict_generator(training_abnormal_coronal_generator)
training_axial_prediction = abnormal_axial.predict_generator(training_abnormal_axial_generator)

validation_sagittal_prediction = abnormal_sagittal.predict_generator(validation_abnormal_sagittal_generator)
validation_coronal_prediction = abnormal_coronal.predict_generator(validation_abnormal_coronal_generator)
validation_axial_prediction = abnormal_axial.predict_generator(validation_abnormal_axial_generator)

training_input = tf.concat([training_sagittal_prediction, training_coronal_prediction, training_axial_prediction],1)
training_output = tf.convert_to_tensor(get_output('abnormal','train'))
validation_input = tf.concat([validation_sagittal_prediction, validation_coronal_prediction, validation_axial_prediction],1)
validation_output = tf.convert_to_tensor(get_output('abnormal','valid'))

abnormal_model = predictor()
abnormal_model_hisory = abnormal_model.fit(training_input, training_output,epochs=30,validation_data=(validation_input,validation_output),batch_size=1)

abnormal_model.save('combine_mrnet_abnormal.h5')

Instructions for updating:
Please use Model.predict, which supports generators.
Epoch 1/30
1017/1017 [==============================] - 2s 2ms/step - loss: 0.4529 - accuracy: 0.7699 - val_loss: 0.4692 - val_accuracy: 0.8053
Epoch 2/30
1017/1017 [==============================] - 2s 2ms/step - loss: 0.3226 - accuracy: 0.8083 - val_loss: 0.4712 - val_accuracy: 0.8053
Epoch 3/30
1017/1017 [==============================] - 2s 2ms/step - loss: 0.2635 - accuracy: 0.8456 - val_loss: 0.4810 - val_accuracy: 0.7876
Epoch 4/30
1017/1017 [==============================] - 2s 2ms/step - loss: 0.2207 - accuracy: 0.9174 - val_loss: 0.4941 - val_accuracy: 0.7965
Epoch 5/30
1017/1017 [==============================] - 2s 2ms/step - loss: 0.1896 - accuracy: 0.9440 - val_loss: 0.5056 - val_accuracy: 0.7876
Epoch 6/30
1017/1017 [==============================] - 2s 2ms/step - loss: 0.1664 - accuracy: 0.9607 - val_loss: 0.5167 - val_accuracy: 0.7876
Epoch 7/30
1017/1017 [==============================] - 

In [ ]:
#load combined model
abnormal_model = predictor()
abnormal_model.load_weights('combine_mrnet_abnormal.h5')

In [ ]:
# testing the model
test_abnormal_sagittal_generator = data_generator('abnormal', 'sagittal', 'test')
test_abnormal_coronal_generator = data_generator('abnormal', 'coronal', 'test')
test_abnormal_axial_generator = data_generator('abnormal', 'axial', 'test')

test_sagittal_prediction = abnormal_sagittal.predict_generator(test_abnormal_sagittal_generator)
test_coronal_prediction = abnormal_coronal.predict_generator(test_abnormal_coronal_generator)
test_axial_prediction = abnormal_axial.predict_generator(test_abnormal_axial_generator)

test_input = tf.concat([test_sagittal_prediction, test_coronal_prediction, test_axial_prediction],1)
test_output = tf.convert_to_tensor(get_output('abnormal','test'))

abnormal_model.evaluate(test_input,test_output,batch_size=1)

120/120 [==============================] - 0s 2ms/step - loss: 0.7697 - accuracy: 0.7917


[0.7696942687034607, 0.7916666865348816]

###ACL

In [ ]:
# training models
acl_sagittal = mrnet_model()
acl_coronal = mrnet_model()
acl_axial = mrnet_model()

training_acl_sagittal_generator = data_generator('acl', 'sagittal','train')
validation_acl_sagittal_generator = data_generator('acl', 'sagittal','valid')
acl_sagittal.history = acl_sagittal.fit_generator(generator=training_acl_sagittal_generator, validation_data=validation_acl_sagittal_generator, epochs=30)

acl_sagittal.save('mrnet_acl_sagittal.h5')

training_acl_axial_generator = data_generator('acl', 'axial','train')
validation_acl_axial_generator = data_generator('acl', 'axial','valid')
acl_axial.history = acl_axial.fit_generator(generator=training_acl_axial_generator, validation_data=validation_acl_axial_generator, epochs=30)

acl_axial.save('mrnet_acl_axial.h5')

training_acl_coronal_generator = data_generator('acl', 'coronal','train')
validation_acl_coronal_generator = data_generator('acl', 'coronal','valid')
acl_coronal.history = acl_coronal.fit_generator(generator=training_acl_coronal_generator, validation_data=validation_acl_coronal_generator, epochs=30)

acl_coronal.save('mrnet_acl_coronal.h5')

In [ ]:
# loading trained models
acl_sagittal = mrnet_model()
acl_coronal = mrnet_model()
acl_axial = mrnet_model()

acl_sagittal.load_weights("mrnet_acl_sagittal.h5")
acl_coronal.load_weights("mrnet_acl_coronal.h5")
acl_axial.load_weights("mrnet_acl_axial.h5")

In [ ]:
# combining models
training_acl_sagittal_generator = data_generator('acl', 'sagittal','train')
validation_acl_sagittal_generator = data_generator('acl', 'sagittal','valid')

training_acl_coronal_generator = data_generator('acl', 'coronal','train')
validation_acl_coronal_generator = data_generator('acl', 'coronal','valid')

training_acl_axial_generator = data_generator('acl', 'axial','train')
validation_acl_axial_generator = data_generator('acl', 'axial','valid')

training_sagittal_prediction = acl_sagittal.predict_generator(training_acl_sagittal_generator)
training_coronal_prediction = acl_coronal.predict_generator(training_acl_coronal_generator)
training_axial_prediction = acl_axial.predict_generator(training_acl_axial_generator)

validation_sagittal_prediction = acl_sagittal.predict_generator(validation_acl_sagittal_generator)
validation_coronal_prediction = acl_coronal.predict_generator(validation_acl_coronal_generator)
validation_axial_prediction = acl_axial.predict_generator(validation_acl_axial_generator)

training_input = tf.concat([training_sagittal_prediction, training_coronal_prediction, training_axial_prediction],1)
training_output = tf.convert_to_tensor(get_output('acl','train'))
validation_input = tf.concat([validation_sagittal_prediction, validation_coronal_prediction, validation_axial_prediction],1)
validation_output = tf.convert_to_tensor(get_output('acl','valid'))

acl_model = predictor()
acl_model_hisory = acl_model.fit(training_input, training_output,epochs=30,validation_data=(validation_input,validation_output),batch_size=1)
acl_model.save('combine_mrnet_acl.h5')

Epoch 1/30
1017/1017 [==============================] - 2s 2ms/step - loss: 0.3786 - accuracy: 0.9725 - val_loss: 0.4302 - val_accuracy: 0.8230
Epoch 2/30
1017/1017 [==============================] - 2s 2ms/step - loss: 0.1885 - accuracy: 0.9833 - val_loss: 0.4228 - val_accuracy: 0.8142
Epoch 3/30
1017/1017 [==============================] - 2s 2ms/step - loss: 0.1236 - accuracy: 0.9931 - val_loss: 0.4317 - val_accuracy: 0.8142
Epoch 4/30
1017/1017 [==============================] - 2s 2ms/step - loss: 0.0917 - accuracy: 0.9980 - val_loss: 0.4428 - val_accuracy: 0.8142
Epoch 5/30
1017/1017 [==============================] - 2s 2ms/step - loss: 0.0729 - accuracy: 0.9990 - val_loss: 0.4543 - val_accuracy: 0.8142
Epoch 6/30
1017/1017 [==============================] - 2s 2ms/step - loss: 0.0605 - accuracy: 1.0000 - val_loss: 0.4652 - val_accuracy: 0.8142
Epoch 7/30
1017/1017 [==============================] - 2s 2ms/step - loss: 0.0518 - accuracy: 1.0000 - val_loss: 0.4753 - val_accuracy:

In [ ]:
#load combined model
acl_model = predictor()
acl_model.load_weights('combine_mrnet_acl.h5')

In [ ]:
# testing the model
test_acl_sagittal_generator = data_generator('acl', 'sagittal', 'test')
test_acl_coronal_generator = data_generator('acl', 'coronal', 'test')
test_acl_axial_generator = data_generator('acl', 'axial', 'test')

test_sagittal_prediction = acl_sagittal.predict_generator(test_acl_sagittal_generator)
test_coronal_prediction = acl_coronal.predict_generator(test_acl_coronal_generator)
test_axial_prediction = acl_axial.predict_generator(test_acl_axial_generator)

test_input = tf.concat([test_sagittal_prediction, test_coronal_prediction, test_axial_prediction],1)
test_output = tf.convert_to_tensor(get_output('acl','test'))

acl_model.evaluate(test_input,test_output,batch_size=1)

###Meniscus

In [ ]:
# training models
meniscus_sagittal = mrnet_model()
meniscus_coronal = mrnet_model()
meniscus_axial = mrnet_model()

training_meniscus_sagittal_generator = data_generator('meniscus', 'sagittal','train')
validation_meniscus_sagittal_generator = data_generator('meniscus', 'sagittal','valid')
meniscus_axial.history = meniscus_sagittal.fit_generator(generator=training_meniscus_sagittal_generator, validation_data=validation_meniscus_sagittal_generator, epochs=30)

meniscus_sagittal.save('mrnet_meniscus_sagittal.h5')

training_meniscus_axial_generator = data_generator('meniscus', 'axial','train')
validation_meniscus_axial_generator = data_generator('meniscus', 'axial','valid')
meniscus_axial.history = meniscus_axial.fit_generator(generator=training_meniscus_axial_generator, validation_data=validation_meniscus_axial_generator, epochs=30)

meniscus_axial.save('mrnet_meniscus_axial.h5')

training_meniscus_coronal_generator = data_generator('meniscus', 'coronal','train')
validation_meniscus_coronal_generator = data_generator('meniscus', 'coronal','valid')
meniscus_coronal.history = meniscus_coronal.fit_generator(generator=training_meniscus_coronal_generator, validation_data=validation_meniscus_coronal_generator, epochs=30)

meniscus_coronal.save('mrnet_meniscus_coronal.h5')

In [ ]:
# loading trained models
meniscus_sagittal = mrnet_model()
meniscus_coronal = mrnet_model()
meniscus_axial = mrnet_model()

meniscus_sagittal.load_weights("mrnet_meniscus_sagittal.h5")
meniscus_coronal.load_weights("mrnet_meniscus_coronal.h5")
meniscus_axial.load_weights("mrnet_meniscus_axial.h5")

In [ ]:
# combining models
training_meniscus_sagittal_generator = data_generator('meniscus', 'sagittal','train')
validation_meniscus_sagittal_generator = data_generator('meniscus', 'sagittal','valid')

training_meniscus_coronal_generator = data_generator('meniscus', 'coronal','train')
validation_meniscus_coronal_generator = data_generator('meniscus', 'coronal','valid')

training_meniscus_axial_generator = data_generator('meniscus', 'axial','train')
validation_meniscus_axial_generator = data_generator('meniscus', 'axial','valid')

training_sagittal_prediction = meniscus_sagittal.predict_generator(training_meniscus_sagittal_generator)
training_coronal_prediction = meniscus_coronal.predict_generator(training_meniscus_coronal_generator)
training_axial_prediction = meniscus_axial.predict_generator(training_meniscus_axial_generator)

validation_sagittal_prediction = meniscus_sagittal.predict_generator(validation_meniscus_sagittal_generator)
validation_coronal_prediction = meniscus_coronal.predict_generator(validation_meniscus_coronal_generator)
validation_axial_prediction = meniscus_axial.predict_generator(validation_meniscus_axial_generator)

training_input = tf.concat([training_sagittal_prediction, training_coronal_prediction, training_axial_prediction],1)
training_output = tf.convert_to_tensor(get_output('meniscus','train'))
validation_input = tf.concat([validation_sagittal_prediction, validation_coronal_prediction, validation_axial_prediction],1)
validation_output = tf.convert_to_tensor(get_output('meniscus','valid'))

meniscus_model = predictor()
meniscus_model_hisory = meniscus_model.fit(training_input, training_output,epochs=30,validation_data=(validation_input,validation_output),batch_size=1)

meniscus_model.save('combine_mrnet_meniscus.h5')

Epoch 1/30
1017/1017 [==============================] - 3s 3ms/step - loss: 0.4783 - accuracy: 0.9154 - val_loss: 0.5866 - val_accuracy: 0.7257
Epoch 2/30
1017/1017 [==============================] - 2s 2ms/step - loss: 0.2396 - accuracy: 0.9961 - val_loss: 0.5885 - val_accuracy: 0.7257
Epoch 3/30
1017/1017 [==============================] - 2s 2ms/step - loss: 0.1549 - accuracy: 0.9990 - val_loss: 0.6102 - val_accuracy: 0.7168
Epoch 4/30
1017/1017 [==============================] - 2s 2ms/step - loss: 0.1137 - accuracy: 1.0000 - val_loss: 0.6340 - val_accuracy: 0.7168
Epoch 5/30
1017/1017 [==============================] - 2s 2ms/step - loss: 0.0896 - accuracy: 1.0000 - val_loss: 0.6570 - val_accuracy: 0.7168
Epoch 6/30
1017/1017 [==============================] - 2s 2ms/step - loss: 0.0739 - accuracy: 1.0000 - val_loss: 0.6791 - val_accuracy: 0.7168
Epoch 7/30
1017/1017 [==============================] - 2s 2ms/step - loss: 0.0628 - accuracy: 1.0000 - val_loss: 0.6993 - val_accuracy:

In [ ]:
#load combined model
meniscus_model = predictor()
meniscus_model.load_weights('combine_mrnet_meniscus.h5')

In [ ]:
# testing the model
test_meniscus_sagittal_generator = data_generator('meniscus', 'sagittal', 'test')
test_meniscus_coronal_generator = data_generator('meniscus', 'coronal', 'test')
test_meniscus_axial_generator = data_generator('meniscus', 'axial', 'test')

test_sagittal_prediction = meniscus_sagittal.predict_generator(test_meniscus_sagittal_generator)
test_coronal_prediction = meniscusl_coronal.predict_generator(test_meniscus_coronal_generator)
test_axial_prediction = meniscus_axial.predict_generator(test_meniscus_axial_generator)

test_input = tf.concat([test_sagittal_prediction, test_coronal_prediction, test_axial_prediction],1)
test_output = tf.convert_to_tensor(get_output('meniscus','test'))

meniscus_model.evaluate(test_input,test_output,batch_size=1)

#Transfer Learning

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
def transfer():
  inputs = keras.Input(shape=(None,256,256,3))

  conv_base = InceptionV3(weights='imagenet',
                  include_top=False,
                  input_shape=(256, 256, 3))(inputs[0])

  avg_layer = layers.GlobalAveragePooling2D()(conv_base)
  avg_layer = tf.expand_dims(avg_layer,0)
  max_layer = layers.GlobalMaxPooling1D()(avg_layer)
  flat = layers.Flatten()(max_layer)
  dense = layers.Dense(512,activation='relu')(flat)
  out = layers.Dense(1, activation='sigmoid')(dense)

  model = models.Model(inputs=inputs, outputs=out)

  model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
  return model

t = transfer()

###Abnormal

In [ ]:
# training models
abnormal_sagittal = transfer()
abnormal_coronal = transfer()
abnormal_axial = transfer()

training_abnormal_sagittal_generator = data_generator('abnormal', 'sagittal','train')
validation_abnormal_sagittal_generator = data_generator('abnormal', 'sagittal','valid')
abnormal_sagittal.history = abnormal_sagittal.fit_generator(generator=training_abnormal_sagittal_generator, validation_data=validation_abnormal_sagittal_generator, epochs=1)

abnormal_sagittal.save('transfer_abnormal_sagittal.h5')

training_abnormal_coronal_generator = data_generator('abnormal', 'coronal','train')
validation_abnormal_coronal_generator = data_generator('abnormal', 'coronal','valid')
abnormal_coronal.history = abnormal_coronal.fit_generator(generator=training_abnormal_coronal_generator, validation_data=validation_abnormal_coronal_generator, epochs=1)

abnormal_coronal.save('transfer_abnormal_coronal.h5')

training_abnormal_axial_generator = data_generator('abnormal', 'axial','train')
validation_abnormal_axial_generator = data_generator('abnormal', 'axial','valid')
abnormal_axial.history = abnormal_axial.fit_generator(generator=training_abnormal_axial_generator, validation_data=validation_abnormal_axial_generator, epochs=1)

abnormal_axial.save('transfer_abnormal_axial.h5')

In [ ]:
# loading trained models
abnormal_sagittal = transfer()
abnormal_coronal = transfer()
abnormal_axial = transfer()

abnormal_sagittal.load_weights("transfer_abnormal_sagittal.h5")
abnormal_coronal.load_weights("transfer_abnormal_coronal.h5")
abnormal_axial.load_weights("transfer_abnormal_axial.h5")

In [ ]:
# combining models
training_abnormal_sagittal_generator = data_generator('abnormal', 'sagittal','train')
validation_abnormal_sagittal_generator = data_generator('abnormal', 'sagittal','valid')

training_abnormal_coronal_generator = data_generator('abnormal', 'coronal','train')
validation_abnormal_coronal_generator = data_generator('abnormal', 'coronal','valid')

training_abnormal_axial_generator = data_generator('abnormal', 'axial','train')
validation_abnormal_axial_generator = data_generator('abnormal', 'axial','valid')

training_sagittal_prediction = abnormal_sagittal.predict_generator(training_abnormal_sagittal_generator)
training_coronal_prediction = abnormal_coronal.predict_generator(training_abnormal_coronal_generator)
training_axial_prediction = abnormal_axial.predict_generator(training_abnormal_axial_generator)

validation_sagittal_prediction = abnormal_sagittal.predict_generator(validation_abnormal_sagittal_generator)
validation_coronal_prediction = abnormal_coronal.predict_generator(validation_abnormal_coronal_generator)
validation_axial_prediction = abnormal_axial.predict_generator(validation_abnormal_axial_generator)

training_input = tf.concat([training_sagittal_prediction, training_coronal_prediction, training_axial_prediction],1)
training_output = tf.convert_to_tensor(get_output('abnormal','train'))
validation_input = tf.concat([validation_sagittal_prediction, validation_coronal_prediction, validation_axial_prediction],1)
validation_output = tf.convert_to_tensor(get_output('abnormal','valid'))

abnormal_model = predictor()
abnormal_model.hisory = abnormal_model.fit(training_input, training_output,epochs=30,validation_data=(validation_input,validation_output),batch_size=1)

abnormal_model.save('combine_transfer_abnormal.h5')

Epoch 1/30
1017/1017 [==============================] - 2s 2ms/step - loss: 0.5218 - accuracy: 0.7847 - val_loss: 0.4970 - val_accuracy: 0.8053
Epoch 2/30
1017/1017 [==============================] - 2s 2ms/step - loss: 0.4952 - accuracy: 0.8083 - val_loss: 0.4973 - val_accuracy: 0.8053
Epoch 3/30
1017/1017 [==============================] - 2s 2ms/step - loss: 0.4944 - accuracy: 0.8083 - val_loss: 0.4968 - val_accuracy: 0.8053
Epoch 4/30
1017/1017 [==============================] - 2s 2ms/step - loss: 0.4931 - accuracy: 0.8083 - val_loss: 0.4949 - val_accuracy: 0.8053
Epoch 5/30
1017/1017 [==============================] - 2s 2ms/step - loss: 0.4921 - accuracy: 0.8083 - val_loss: 0.4950 - val_accuracy: 0.8053
Epoch 6/30
1017/1017 [==============================] - 2s 2ms/step - loss: 0.4916 - accuracy: 0.8083 - val_loss: 0.4938 - val_accuracy: 0.8053
Epoch 7/30
1017/1017 [==============================] - 2s 2ms/step - loss: 0.4900 - accuracy: 0.8083 - val_loss: 0.4941 - val_accuracy:

In [ ]:
#load combined model
abnormal_model = predictor()
abnormal_model.load_weights('combine_transfer_abnormal.h5')

In [ ]:
# testing the model
test_abnormal_sagittal_generator = data_generator('abnormal', 'sagittal', 'test')
test_abnormal_coronal_generator = data_generator('abnormal', 'coronal', 'test')
test_abnormal_axial_generator = data_generator('abnormal', 'axial', 'test')

test_sagittal_prediction = abnormal_sagittal.predict_generator(test_abnormal_sagittal_generator)
test_coronal_prediction = abnormal_coronal.predict_generator(test_abnormal_coronal_generator)
test_axial_prediction = abnormal_axial.predict_generator(test_abnormal_axial_generator)

test_input = tf.concat([test_sagittal_prediction, test_coronal_prediction, test_axial_prediction],1)
test_output = tf.convert_to_tensor(get_output('abnormal','test'))

abnormal_model.evaluate(test_input,test_output,batch_size=1)

120/120 [==============================] - 0s 2ms/step - loss: 0.5051 - accuracy: 0.7917


[0.5050693154335022, 0.7916666865348816]

###ACL

In [ ]:
# training models
acl_sagittal = transfer()
acl_coronal = transfer()
acl_axial = transfer()

training_acl_sagittal_generator = data_generator('acl', 'sagittal','train')
validation_acl_sagittal_generator = data_generator('acl', 'sagittal','valid')
acl_sagittal.history = acl_sagittal.fit_generator(generator=training_acl_sagittal_generator, validation_data=validation_acl_sagittal_generator, epochs=1)

acl_sagittal.save('transfer_acl_sagittal.h5')

training_acl_axial_generator = data_generator('acl', 'axial','train')
validation_acl_axial_generator = data_generator('acl', 'axial','valid')
acl_axial.history = acl_axial.fit_generator(generator=training_acl_axial_generator, validation_data=validation_acl_axial_generator, epochs=1)

acl_axial.save('transfer_acl_axial.h5')

training_acl_coronal_generator = data_generator('acl', 'coronal','train')
validation_acl_coronal_generator = data_generator('acl', 'coronal','valid')
acl_coronal.history = acl_coronal.fit_generator(generator=training_acl_coronal_generator, validation_data=validation_acl_coronal_generator, epochs=1)

acl_coronal.save('transfer_acl_coronal.h5')

In [ ]:
# loading trained models
acl_sagittal = transfer()
acl_coronal = transfer()
acl_axial = transfer()

acl_sagittal.load_weights("transfer_acl_sagittal.h5")
acl_coronal.load_weights("transfer_acl_coronal.h5")
acl_axial.load_weights("transfer_acl_axial.h5")

In [ ]:
# combining models
training_acl_sagittal_generator = data_generator('acl', 'sagittal','train')
validation_acl_sagittal_generator = data_generator('acl', 'sagittal','valid')

training_acl_coronal_generator = data_generator('acl', 'coronal','train')
validation_acl_coronal_generator = data_generator('acl', 'coronal','valid')

training_acl_axial_generator = data_generator('acl', 'axial','train')
validation_acl_axial_generator = data_generator('acl', 'axial','valid')

training_sagittal_prediction = acl_sagittal.predict_generator(training_acl_sagittal_generator)
training_coronal_prediction = acl_coronal.predict_generator(training_acl_coronal_generator)
training_axial_prediction = acl_axial.predict_generator(training_acl_axial_generator)

validation_sagittal_prediction = acl_sagittal.predict_generator(validation_acl_sagittal_generator)
validation_coronal_prediction = acl_coronal.predict_generator(validation_acl_coronal_generator)
validation_axial_prediction = acl_axial.predict_generator(validation_acl_axial_generator)

training_input = tf.concat([training_sagittal_prediction, training_coronal_prediction, training_axial_prediction],1)
training_output = tf.convert_to_tensor(get_output('acl','train'))
validation_input = tf.concat([validation_sagittal_prediction, validation_coronal_prediction, validation_axial_prediction],1)
validation_output = tf.convert_to_tensor(get_output('acl','valid'))

acl_model = predictor()
acl_model.hisory = acl_model.fit(training_input, training_output,epochs=30,validation_data=(validation_input,validation_output),batch_size=1)

acl_model.save('combine_transfer_acl.h5')

In [ ]:
#load combined model
acl_model = predictor()
acl_model.load_weights('combine_transfer_acl.h5')

In [ ]:
# testing the model
test_acl_sagittal_generator = data_generator('acl', 'sagittal', 'test')
test_acl_coronal_generator = data_generator('acl', 'coronal', 'test')
test_acl_axial_generator = data_generator('acl', 'axial', 'test')

test_sagittal_prediction = acl_sagittal.predict_generator(test_acl_sagittal_generator)
test_coronal_prediction = acl_coronal.predict_generator(test_acl_coronal_generator)
test_axial_prediction = acl_axial.predict_generator(test_acl_axial_generator)

test_input = tf.concat([test_sagittal_prediction, test_coronal_prediction, test_axial_prediction],1)
test_output = tf.convert_to_tensor(get_output('acl','test'))

acl_model.evaluate(test_input,test_output,batch_size=1)

120/120 [==============================] - 0s 2ms/step - loss: 0.8596 - accuracy: 0.5500


[0.8596245050430298, 0.550000011920929]

###Meniscus

In [ ]:
# training models
meniscus_sagittal = transfer()
meniscus_coronal = transfer()
meniscus_axial = transfer()

training_meniscus_sagittal_generator = data_generator('meniscus', 'sagittal','train')
validation_meniscus_sagittal_generator = data_generator('meniscus', 'sagittal','valid')
meniscus_axial.history = meniscus_sagittal.fit_generator(generator=training_meniscus_sagittal_generator, validation_data=validation_meniscus_sagittal_generator, epochs=1)

meniscus_sagittal.save('transfer_meniscus_sagittal.h5')

training_meniscus_axial_generator = data_generator('meniscus', 'axial','train')
validation_meniscus_axial_generator = data_generator('meniscus', 'axial','valid')
meniscus_axial.history = meniscus_axial.fit_generator(generator=training_meniscus_axial_generator, validation_data=validation_meniscus_axial_generator, epochs=1)

meniscus_axial.save('transfer_meniscus_axial.h5')

training_meniscus_coronal_generator = data_generator('meniscus', 'coronal','train')
validation_meniscus_coronal_generator = data_generator('meniscus', 'coronal','valid')
meniscus_coronal.history = meniscus_coronal.fit_generator(generator=training_meniscus_coronal_generator, validation_data=validation_meniscus_coronal_generator, epochs=1)

meniscus_coronal.save('transfer_meniscus_coronal.h5')

1017/1017 [==============================] - 304s 299ms/step - loss: 0.6852 - accuracy: 0.6313 - val_loss: 0.6659 - val_accuracy: 0.6283


In [ ]:
# loading trained models
meniscus_sagittal = transfer()
meniscus_coronal = transfer()
meniscus_axial = transfer()

meniscus_sagittal.load_weights("transfer_meniscus_sagittal.h5")
meniscus_coronal.load_weights("transfer_meniscus_coronal.h5")
meniscus_axial.load_weights("transfer_meniscus_axial.h5")

In [ ]:
# combining models
training_meniscus_sagittal_generator = data_generator('meniscus', 'sagittal','train')
validation_meniscus_sagittal_generator = data_generator('meniscus', 'sagittal','valid')

training_meniscus_coronal_generator = data_generator('meniscus', 'coronal','train')
validation_meniscus_coronal_generator = data_generator('meniscus', 'coronal','valid')

training_meniscus_axial_generator = data_generator('meniscus', 'axial','train')
validation_meniscus_axial_generator = data_generator('meniscus', 'axial','valid')

training_sagittal_prediction = meniscus_sagittal.predict_generator(training_meniscus_sagittal_generator)
training_coronal_prediction = meniscus_coronal.predict_generator(training_meniscus_coronal_generator)
training_axial_prediction = meniscus_axial.predict_generator(training_meniscus_axial_generator)

validation_sagittal_prediction = meniscus_sagittal.predict_generator(validation_meniscus_sagittal_generator)
validation_coronal_prediction = meniscus_coronal.predict_generator(validation_meniscus_coronal_generator)
validation_axial_prediction = meniscus_axial.predict_generator(validation_meniscus_axial_generator)

training_input = tf.concat([training_sagittal_prediction, training_coronal_prediction, training_axial_prediction],1)
training_output = tf.convert_to_tensor(get_output('meniscus','train'))
validation_input = tf.concat([validation_sagittal_prediction, validation_coronal_prediction, validation_axial_prediction],1)
validation_output = tf.convert_to_tensor(get_output('meniscus','valid'))

meniscus_model = predictor()
meniscus_model.hisory = meniscus_model.fit(training_input, training_output,epochs=100,validation_data=(validation_input,validation_output),batch_size=1)

meniscus_model.save('combine_transfer_meniscus.h5')

Epoch 1/100
1017/1017 [==============================] - 2s 2ms/step - loss: 0.6698 - accuracy: 0.5998 - val_loss: 0.6403 - val_accuracy: 0.6637
Epoch 2/100
1017/1017 [==============================] - 2s 2ms/step - loss: 0.6489 - accuracy: 0.6470 - val_loss: 0.6401 - val_accuracy: 0.6637
Epoch 3/100
1017/1017 [==============================] - 2s 2ms/step - loss: 0.6479 - accuracy: 0.6470 - val_loss: 0.6371 - val_accuracy: 0.6637
Epoch 4/100
1017/1017 [==============================] - 2s 2ms/step - loss: 0.6458 - accuracy: 0.6470 - val_loss: 0.6350 - val_accuracy: 0.6637
Epoch 5/100
1017/1017 [==============================] - 2s 2ms/step - loss: 0.6445 - accuracy: 0.6470 - val_loss: 0.6338 - val_accuracy: 0.6637
Epoch 6/100
1017/1017 [==============================] - 2s 2ms/step - loss: 0.6442 - accuracy: 0.6470 - val_loss: 0.6328 - val_accuracy: 0.6637
Epoch 7/100
1017/1017 [==============================] - 2s 2ms/step - loss: 0.6427 - accuracy: 0.6470 - val_loss: 0.6324 - val_ac

In [ ]:
#load combined model
meniscus_model = predictor()
meniscus_model.load_weights('combine_transfer_meniscus.h5')

In [ ]:
# testing the model
test_meniscus_sagittal_generator = data_generator('meniscus', 'sagittal', 'test')
test_meniscus_coronal_generator = data_generator('meniscus', 'coronal', 'test')
test_meniscus_axial_generator = data_generator('meniscus', 'axial', 'test')

test_sagittal_prediction = meniscus_sagittal.predict_generator(test_meniscus_sagittal_generator)
test_coronal_prediction = meniscus_coronal.predict_generator(test_meniscus_coronal_generator)
test_axial_prediction = meniscus_axial.predict_generator(test_meniscus_axial_generator)

test_input = tf.concat([test_sagittal_prediction, test_coronal_prediction, test_axial_prediction],1)
test_output = tf.convert_to_tensor(get_output('meniscus','test'))

meniscus_model.evaluate(test_input,test_output,batch_size=1)

120/120 [==============================] - 0s 1ms/step - loss: 0.6867 - accuracy: 0.5583


[0.6867029070854187, 0.5583333373069763]

#Contribution

##Abnormal

In [ ]:
sagittal_vgg = vgg_model()
sagittal_resnet = ResNet()
sagittal_inception = inception_v3()
sagittal_vgg.load_weights('vgg_abnormal_sagittal.h5')
sagittal_resnet.load_weights('resnet_abnormal_sagittal.h5')
sagittal_inception.load_weights('inception_abnormal_sagittal.h5')

coronal_vgg = vgg_model()
coronal_resnet = ResNet()
coronal_inception = inception_v3()
coronal_vgg.load_weights('vgg_abnormal_coronal.h5')
coronal_resnet.load_weights('resnet_abnormal_coronal.h5')
coronal_inception.load_weights('inception_abnormal_coronal.h5')

axial_vgg = vgg_model()
axial_resnet = ResNet()
axial_inception = inception_v3()
axial_vgg.load_weights('vgg_abnormal_axial.h5')
axial_resnet.load_weights('resnet_abnormal_axial.h5')
axial_inception.load_weights('inception_abnormal_axial.h5')

In [ ]:
training_ sagittal_generator = data_generator('abnormal', 'sagittal','train')
validation_sagittal_generator = data_generator('abnormal', 'sagittal','valid')

training_vgg_prediction = sagittal_vgg.predict_generator(training_sagittal_generator)
training_resnet_prediction = sagittal_resnet.predict_generator(training_sagittal_generator)
training_inception_prediction = sagittal_inception.predict_generator(ttraining_sagittal_generator)

validation_vgg_prediction = sagittal_vgg.predict_generator(validation_sagittal_generator)
validation_resnet_prediction = sagittal_resnet.predict_generator(validation_sagittal_generator)
validation_inception_prediction = sagittal_inception.predict_generator(validation_sagittal_generator)

training_input = tf.concat([training_vgg_prediction, training_resnet_prediction, training_inception_prediction],1)
training_output = tf.convert_to_tensor(get_output('abnormal','train'))
validation_input = tf.concat([validation_vgg_prediction, validation_resnet_prediction, validation_inception_prediction],1)
validation_output = tf.convert_to_tensor(get_output('abnormal','valid'))

sagittal_model = predictor()
sagittal_model.fit(training_input, training_output,epochs=30,validation_data=(validation_input,validation_output),batch_size=1)

training_sagittal_prediction = sagittal_model.predict(training_input)
validation_sagittal_prediction = sagittal_model.predict(validation_input)

In [ ]:
training_coronal_generator = data_generator('abnormal', 'coronal','train')
validation_coronal_generator = data_generator('abnormal', 'coronal','valid')

training_vgg_prediction = coronal_vgg.predict_generator(training_coronal_generator)
training_resnet_prediction = coronal_resnet.predict_generator(training_coronal_generator)
training_inception_prediction = coronal_inception.predict_generator(ttraining_coronal_generator)

validation_vgg_prediction = coronal_vgg.predict_generator(validation_coronal_generator)
validation_resnet_prediction = coronal_resnet.predict_generator(validation_coronal_generator)
validation_inception_prediction = coronal_inception.predict_generator(validation_coronal_generator)

training_input = tf.concat([training_vgg_prediction, training_resnet_prediction, training_inception_prediction],1)
training_output = tf.convert_to_tensor(get_output('abnormal','train'))
validation_input = tf.concat([validation_vgg_prediction, validation_resnet_prediction, validation_inception_prediction],1)
validation_output = tf.convert_to_tensor(get_output('abnormal','valid'))

coronal_model = predictor()
coronal_model.fit(training_input, training_output,epochs=30,validation_data=(validation_input,validation_output),batch_size=1)

training_coronal_prediction = coronal_model.predict(training_input)
validation_coronal_prediction = coronal_model.predict(validation_input)

In [ ]:
training_axial_generator = data_generator('abnormal', 'axial','train')
validation_axial_generator = data_generator('abnormal', 'axial','valid')

training_vgg_prediction = axial_vgg.predict_generator(training_axial_generator)
training_resnet_prediction = axial_resnet.predict_generator(training_axial_generator)
training_inception_prediction = axial_inception.predict_generator(ttraining_axial_generator)

validation_vgg_prediction = axial_vgg.predict_generator(validation_axial_generator)
validation_resnet_prediction = axial_resnet.predict_generator(validation_axial_generator)
validation_inception_prediction = axial_inception.predict_generator(validation_axial_generator)

training_input = tf.concat([training_vgg_prediction, training_resnet_prediction, training_inception_prediction],1)
training_output = tf.convert_to_tensor(get_output('abnormal','train'))
validation_input = tf.concat([validation_vgg_prediction, validation_resnet_prediction, validation_inception_prediction],1)
validation_output = tf.convert_to_tensor(get_output('abnormal','valid'))

axial_model = predictor()
axial_model.fit(training_input, training_output,epochs=30,validation_data=(validation_input,validation_output),batch_size=1)

training_axial_prediction = axial_model.predict(training_input)
validation_axial_prediction = axial_model.predict(validation_input)

In [ ]:
training_input = tf.concat([training_sagittal_prediction, training_coronal_prediction, training_axial_prediction],1)
training_output = tf.convert_to_tensor(get_output('abnormal','train'))
validation_input = tf.concat([validation_sagittal_prediction, validation_coronal_prediction, validation_axial_prediction],1)
validation_output = tf.convert_to_tensor(get_output('abnormal','valid'))

model = predictor()
model.fit(training_input, training_output,epochs=30,validation_data=(validation_input,validation_output),batch_size=1)
model.save('contribution_abnormal.h5')

##ACL

In [ ]:
sagittal_vgg = vgg_model()
sagittal_resnet = ResNet()
sagittal_inception = inception_v3()
sagittal_vgg.load_weights('vgg_acl_sagittal.h5')
sagittal_resnet.load_weights('resnet_acl_sagittal.h5')
sagittal_inception.load_weights('inception_acl_sagittal.h5')

coronal_vgg = vgg_model()
coronal_resnet = ResNet()
coronal_inception = inception_v3()
coronal_vgg.load_weights('vgg_acl_coronal.h5')
coronal_resnet.load_weights('resnet_acl_coronal.h5')
coronal_inception.load_weights('inception_acl_coronal.h5')

axial_vgg = vgg_model()
axial_resnet = ResNet()
axial_inception = inception_v3()
axial_vgg.load_weights('vgg_acl_axial.h5')
axial_resnet.load_weights('resnet_acl_axial.h5')
axial_inception.load_weights('inception_acl_axial.h5')

In [ ]:
training_ sagittal_generator = data_generator('acl', 'sagittal','train')
validation_sagittal_generator = data_generator('acl', 'sagittal','valid')

training_vgg_prediction = sagittal_vgg.predict_generator(training_sagittal_generator)
training_resnet_prediction = sagittal_resnet.predict_generator(training_sagittal_generator)
training_inception_prediction = sagittal_inception.predict_generator(ttraining_sagittal_generator)

validation_vgg_prediction = sagittal_vgg.predict_generator(validation_sagittal_generator)
validation_resnet_prediction = sagittal_resnet.predict_generator(validation_sagittal_generator)
validation_inception_prediction = sagittal_inception.predict_generator(validation_sagittal_generator)

training_input = tf.concat([training_vgg_prediction, training_resnet_prediction, training_inception_prediction],1)
training_output = tf.convert_to_tensor(get_output('acl','train'))
validation_input = tf.concat([validation_vgg_prediction, validation_resnet_prediction, validation_inception_prediction],1)
validation_output = tf.convert_to_tensor(get_output('acl','valid'))

sagittal_model = predictor()
sagittal_model.fit(training_input, training_output,epochs=30,validation_data=(validation_input,validation_output),batch_size=1)

training_sagittal_prediction = sagittal_model.predict(training_input)
validation_sagittal_prediction = sagittal_model.predict(validation_input)

In [ ]:
training_coronal_generator = data_generator('acl', 'coronal','train')
validation_coronal_generator = data_generator('acl', 'coronal','valid')

training_vgg_prediction = coronal_vgg.predict_generator(training_coronal_generator)
training_resnet_prediction = coronal_resnet.predict_generator(training_coronal_generator)
training_inception_prediction = coronal_inception.predict_generator(ttraining_coronal_generator)

validation_vgg_prediction = coronal_vgg.predict_generator(validation_coronal_generator)
validation_resnet_prediction = coronal_resnet.predict_generator(validation_coronal_generator)
validation_inception_prediction = coronal_inception.predict_generator(validation_coronal_generator)

training_input = tf.concat([training_vgg_prediction, training_resnet_prediction, training_inception_prediction],1)
training_output = tf.convert_to_tensor(get_output('acl','train'))
validation_input = tf.concat([validation_vgg_prediction, validation_resnet_prediction, validation_inception_prediction],1)
validation_output = tf.convert_to_tensor(get_output('acl','valid'))

coronal_model = predictor()
coronal_model.fit(training_input, training_output,epochs=30,validation_data=(validation_input,validation_output),batch_size=1)

training_coronal_prediction = coronal_model.predict(training_input)
validation_coronal_prediction = coronal_model.predict(validation_input)

In [ ]:
training_axial_generator = data_generator('acl', 'axial','train')
validation_axial_generator = data_generator('acl', 'axial','valid')

training_vgg_prediction = axial_vgg.predict_generator(training_axial_generator)
training_resnet_prediction = axial_resnet.predict_generator(training_axial_generator)
training_inception_prediction = axial_inception.predict_generator(ttraining_axiall_generator)

validation_vgg_prediction = axial_vgg.predict_generator(validation_axial_generator)
validation_resnet_prediction = axial_resnet.predict_generator(validation_axial_generator)
validation_inception_prediction = axial_inception.predict_generator(validation_axiall_generator)

training_input = tf.concat([training_vgg_prediction, training_resnet_prediction, training_inception_prediction],1)
training_output = tf.convert_to_tensor(get_output('acl','train'))
validation_input = tf.concat([validation_vgg_prediction, validation_resnet_prediction, validation_inception_prediction],1)
validation_output = tf.convert_to_tensor(get_output('acl','valid'))

axial_model = predictor()
axial_model.fit(training_input, training_output,epochs=30,validation_data=(validation_input,validation_output),batch_size=1)

training_axial_prediction = axial_model.predict(training_input)
validation_axial_prediction = axial_model.predict(validation_input)

In [ ]:
training_input = tf.concat([training_sagittal_prediction, training_coronal_prediction, training_axial_prediction],1)
training_output = tf.convert_to_tensor(get_output('acl','train'))
validation_input = tf.concat([validation_sagittal_prediction, validation_coronal_prediction, validation_axial_prediction],1)
validation_output = tf.convert_to_tensor(get_output('acl','valid'))

model = predictor()
model.fit(training_input, training_output,epochs=30,validation_data=(validation_input,validation_output),batch_size=1)
model.save('contribution_acl.h5')

##Meniscus

In [ ]:
sagittal_vgg = vgg_model()
sagittal_resnet = ResNet()
sagittal_inception = inception_v3()
sagittal_vgg.load_weights('vgg_meniscus_sagittal.h5')
sagittal_resnet.load_weights('resnet_meniscus_sagittal.h5')
sagittal_inception.load_weights('inception_meniscus_sagittal.h5')

coronal_vgg = vgg_model()
coronal_resnet = ResNet()
coronal_inception = inception_v3()
coronal_vgg.load_weights('vgg_meniscus_coronal.h5')
coronal_resnet.load_weights('resnet_meniscus_coronal.h5')
coronal_inception.load_weights('inception_meniscus_coronal.h5')

axial_vgg = vgg_model()
axial_resnet = ResNet()
axial_inception = inception_v3()
axial_vgg.load_weights('vgg_meniscus_axial.h5')
axial_resnet.load_weights('resnet_meniscus_axial.h5')
axial_inception.load_weights('inception_meniscus_axial.h5'

In [ ]:
training_ sagittal_generator = data_generator('meniscus', 'sagittal','train')
validation_sagittal_generator = data_generator('meniscus', 'sagittal','valid')

training_vgg_prediction = sagittal_vgg.predict_generator(training_sagittal_generator)
training_resnet_prediction = sagittal_resnet.predict_generator(training_sagittal_generator)
training_inception_prediction = sagittal_inception.predict_generator(ttraining_sagittal_generator)

validation_vgg_prediction = sagittal_vgg.predict_generator(validation_sagittal_generator)
validation_resnet_prediction = sagittal_resnet.predict_generator(validation_sagittal_generator)
validation_inception_prediction = sagittal_inception.predict_generator(validation_sagittal_generator)

training_input = tf.concat([training_vgg_prediction, training_resnet_prediction, training_inception_prediction],1)
training_output = tf.convert_to_tensor(get_output('meniscus','train'))
validation_input = tf.concat([validation_vgg_prediction, validation_resnet_prediction, validation_inception_prediction],1)
validation_output = tf.convert_to_tensor(get_output('meniscus','valid'))

sagittal_model = predictor()
sagittal_model.fit(training_input, training_output,epochs=30,validation_data=(validation_input,validation_output),batch_size=1)

training_sagittal_prediction = sagittal_model.predict(training_input)
validation_sagittal_prediction = sagittal_model.predict(validation_input)

In [ ]:
training_coronal_generator = data_generator('meniscus', 'coronal','train')
validation_coronal_generator = data_generator('meniscus', 'coronal','valid')

training_vgg_prediction = coronal_vgg.predict_generator(training_coronal_generator)
training_resnet_prediction = coronal_resnet.predict_generator(training_coronal_generator)
training_inception_prediction = coronal_inception.predict_generator(ttraining_coronal_generator)

validation_vgg_prediction = coronal_vgg.predict_generator(validation_coronal_generator)
validation_resnet_prediction = coronal_resnet.predict_generator(validation_coronal_generator)
validation_inception_prediction = coronal_inception.predict_generator(validation_coronal_generator)

training_input = tf.concat([training_vgg_prediction, training_resnet_prediction, training_inception_prediction],1)
training_output = tf.convert_to_tensor(get_output('meniscus','train'))
validation_input = tf.concat([validation_vgg_prediction, validation_resnet_prediction, validation_inception_prediction],1)
validation_output = tf.convert_to_tensor(get_output('meniscus','valid'))

coronal_model = predictor()
coronal_model.fit(training_input, training_output,epochs=30,validation_data=(validation_input,validation_output),batch_size=1)

training_coronal_prediction = coronal_model.predict(training_input)
validation_coronal_prediction = coronal_model.predict(validation_input)

In [ ]:
training_axial_generator = data_generator('meniscus', 'axial','train')
validation_axial_generator = data_generator('meniscus', 'axial','valid')

training_vgg_prediction = axial_vgg.predict_generator(training_axial_generator)
training_resnet_prediction = axial_resnet.predict_generator(training_axial_generator)
training_inception_prediction = axial_inception.predict_generator(ttraining_axiall_generator)

validation_vgg_prediction = axial_vgg.predict_generator(validation_axial_generator)
validation_resnet_prediction = axial_resnet.predict_generator(validation_axial_generator)
validation_inception_prediction = axial_inception.predict_generator(validation_axiall_generator)

training_input = tf.concat([training_vgg_prediction, training_resnet_prediction, training_inception_prediction],1)
training_output = tf.convert_to_tensor(get_output('meniscus','train'))
validation_input = tf.concat([validation_vgg_prediction, validation_resnet_prediction, validation_inception_prediction],1)
validation_output = tf.convert_to_tensor(get_output('meniscus','valid'))

axial_model = predictor()
axial_model.fit(training_input, training_output,epochs=30,validation_data=(validation_input,validation_output),batch_size=1)

training_axial_prediction = axial_model.predict(training_input)
validation_axial_prediction = axial_model.predict(validation_input)

In [ ]:
training_input = tf.concat([training_sagittal_prediction, training_coronal_prediction, training_axial_prediction],1)
training_output = tf.convert_to_tensor(get_output('meniscus','train'))
validation_input = tf.concat([validation_sagittal_prediction, validation_coronal_prediction, validation_axial_prediction],1)
validation_output = tf.convert_to_tensor(get_output('meniscus','valid'))

model = predictor()
model.fit(training_input, training_output,epochs=30,validation_data=(validation_input,validation_output),batch_size=1)
model.save('contribution_meniscus.h5')

#Failed attempt 1

In [ ]:
#Data generator
class data_generator_multiple(keras.utils.Sequence):
    def __init__(self, injury, data_type):
        self.injury = injury
        self.data_type = data_type
        if(data_type == 'train'):
          self.labels = pd.read_csv('train-{}.csv'.format(self.injury), names=['case', 'label'], header=None,
                              dtype={'case': str, 'label': np.int64})['label'].tolist()
          self.case_list = pd.read_csv('train-{}.csv'.format(self.injury), names=['case', 'label'], header=None,
                               dtype={'case': str, 'label': np.int64})['case'].tolist()
          self.labels = self.labels[0:1017]
          self.case_list = self.case_list[0:1017]
          self.data_path = 'train'
        elif(data_type == 'valid'):
          self.labels = pd.read_csv('train-{}.csv'.format(self.injury), names=['case', 'label'], header=None,
                              dtype={'case': str, 'label': np.int64})['label'].tolist()
          self.case_list = pd.read_csv('train-{}.csv'.format(self.injury), names=['case', 'label'], header=None,
                               dtype={'case': str, 'label': np.int64})['case'].tolist()
          self.labels = self.labels[1017:1130]
          self.case_list = self.case_list[1017:1130]
          self.data_path = 'train'
        else:
          self.labels = pd.read_csv('valid-{}.csv'.format(self.injury), names=['case', 'label'], header=None,
                              dtype={'case': str, 'label': np.int64})['label'].tolist()
          self.case_list = pd.read_csv('valid-{}.csv'.format(self.injury), names=['case', 'label'], header=None,
                               dtype={'case': str, 'label': np.int64})['case'].tolist()
          self.data_path = 'valid'

    def __len__(self):
        return len(self.case_list)

    def __getitem__(self, index):
        fpath = '{}/{}/{}.npy'.format(self.data_path, 'sagittal', self.case_list[index])
        stack1 = np.load(fpath).astype(float)
        for i in range(stack1.shape[0]):
          stack1[i] = stack1[i]/255.
        stack1 = np.repeat(stack1[:, :, :, np.newaxis], 3, axis=3)
        stack1 = np.expand_dims(stack1, axis=0)

        fpath = '{}/{}/{}.npy'.format(self.data_path, 'coronal', self.case_list[index])
        stack2 = np.load(fpath).astype(float)
        for i in range(stack2.shape[0]):
          stack2[i] = stack2[i]/255.
        stack2 = np.repeat(stack2[:, :, :, np.newaxis], 3, axis=3)
        stack2 = np.expand_dims(stack2, axis=0)

        fpath = '{}/{}/{}.npy'.format(self.data_path, 'axial', self.case_list[index])
        stack3 = np.load(fpath).astype(float)
        for i in range(stack3.shape[0]):
          stack3[i] = stack3[i]/255.
        stack3 = np.repeat(stack3[:, :, :, np.newaxis], 3, axis=3)
        stack3 = np.expand_dims(stack3, axis=0)


        y = self.labels[index]
        y = np.array(y).reshape(1,1)
        yield(stack1, stack2, stack3, y)

In [ ]:
def load_one_stack(case, data_path, plane='coronal'):
    fpath = '{}/{}/{}.npy'.format(data_path, plane, case)
    stack = np.load(fpath).astype(float)
    for i in range(stack.shape[0]):
      stack[i] = stack[i]/255.

    return np.repeat(stack[:, :, :, np.newaxis], 3, axis=3)

def load_stacks(case, data_path):
    x = {}
    planes = ['coronal', 'sagittal', 'axial']
    for i, plane in enumerate(planes):

        x[plane] = load_one_stack(case, data_path, plane=plane)
    return x

def load_cases(train=True, n=30, index=0):
    assert (type(n) == int) and (n < 1250)
    if train:
        case_list = pd.read_csv('train-acl.csv', names=['case', 'label'], header=None,
                               dtype={'case': str, 'label': np.int64})['case'].tolist()
        y_acl =  pd.read_csv('train-acl.csv', names=['case', 'label'], header=None,
                              dtype={'case': str, 'label': np.int64})['label'].tolist()
        y_meniscus =  pd.read_csv('train-meniscus.csv', names=['case', 'label'], header=None,
                              dtype={'case': str, 'label': np.int64})['label'].tolist()
        y_abnormal =  pd.read_csv('train-abnormal.csv', names=['case', 'label'], header=None,
                              dtype={'case': str, 'label': np.int64})['label'].tolist()       
    else:
        case_list = pd.read_csv('valid-acl.csv', names=['case', 'label'], header=None,
                               dtype={'case': str, 'label': np.int64})['case'].tolist() 
        y_acl =  pd.read_csv('valid-acl.csv', names=['case', 'label'], header=None,
                               dtype={'case': str, 'label': np.int64})['label'].tolist()                           
        y_meniscus =  pd.read_csv('valid-meniscus.csv', names=['case', 'label'], header=None,
                              dtype={'case': str, 'label': np.int64})['label'].tolist()
        y_abnormal =  pd.read_csv('valid-abnormal.csv', names=['case', 'label'], header=None,
                              dtype={'case': str, 'label': np.int64})['label'].tolist()
    cases = {}
    if n is not None:
        case_list = case_list[index*n : (index+1)*n]
        y_acl = y_acl[index*n : (index+1)*n]
        y_meniscus = y_meniscus[index*n : (index+1)*n]
        y_abnormal = y_abnormal[index*n : (index+1)*n]
        
    for case in tqdm_notebook(case_list, leave=False):
      if train:
        x = load_stacks(case, 'train')
      else: 
        x = load_stacks(case, 'valid')
        
      cases[case] = x
    
    percent_train = (int)(0.9*len(cases))
    if train:
      return dict(list(cases.items())[:percent_train]), dict(list(cases.items())[percent_train:]) ,
      y_acl, y_meniscus, y_abnormal
    else:
      return cases , y_acl, y_meniscus, y_abnormal

In [ ]:
def free_ram(x_train, x_valid):
  del x_train
  del x_valid

# free_ram()

#Failed attempt 2

In [ ]:
model_axial_abnormal = models.Sequential()
model_axial_abnormal.add(layers.Flatten(input_shape=[1, 1, 512]))
model_axial_abnormal.add(layers.Dense(512, activation='relu'))
model_axial_abnormal.add(layers.Dense(1, activation='sigmoid'))

model_axial_abnormal.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

#model_axial_abnormal.summary()

model_coronal_abnormal = models.Sequential()
model_coronal_abnormal.add(layers.Flatten(input_shape=[1, 1, 512]))
model_coronal_abnormal.add(layers.Dense(512, activation='relu'))
model_coronal_abnormal.add(layers.Dense(1, activation='sigmoid'))

model_coronal_abnormal.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

#model_coronal_abnormal.summary()

model_sagittal_abnormal = models.Sequential()
model_sagittal_abnormal.add(layers.Flatten(input_shape=[1, 1, 512]))
model_sagittal_abnormal.add(layers.Dense(512, activation='relu'))
model_sagittal_abnormal.add(layers.Dense(1, activation='sigmoid'))

model_sagittal_abnormal.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

#model_sagittal_abnormal.summary()

model_abnormal = models.Sequential()
model_abnormal.add(keras.Input(shape=(3,)))
model_abnormal.add(layers.Dense(1, activation='sigmoid'))

model_abnormal.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

#model_abnormal.summary()

In [ ]:
#Model Training
n = 30
percent_train = (int)(0.9*n)

for i in range (20):
  print (i)

  #data generation
  x_train, x_valid, y_train_acl, y_train_meniscus, y_train_abnormal = load_cases(index=i)
  y_train_acl, y_valid_acl = y_train_acl[:percent_train], y_train_acl[percent_train:]
  y_train_meniscus, y_valid_meniscus = y_train_meniscus[:percent_train], y_train_meniscus[percent_train:]
  y_train_abnormal, y_valid_abnormal = y_train_abnormal[:percent_train], y_train_abnormal[percent_train:]

  #prediction
  x_train_vgg_axial = []
  x_train_vgg_coronal = []
  x_train_vgg_sagittal = []
  for case in x_train:
    x_train_vgg_axial.append(tf.reduce_max(VGGmodel.predict(x_train[case]['axial']),0))
    x_train_vgg_coronal.append(tf.reduce_max(VGGmodel.predict(x_train[case]['coronal']),0))
    x_train_vgg_sagittal.append(tf.reduce_max(VGGmodel.predict(x_train[case]['sagittal']),0))

  x_valid_vgg_axial = []
  x_valid_vgg_coronal = []
  x_valid_vgg_sagittal = []
  for case in x_valid:
    x_valid_vgg_axial.append(tf.reduce_max(VGGmodel.predict(x_valid[case]['axial']),0))
    x_valid_vgg_coronal.append(tf.reduce_max(VGGmodel.predict(x_valid[case]['coronal']),0))
    x_valid_vgg_sagittal.append(tf.reduce_max(VGGmodel.predict(x_valid[case]['sagittal']),0))

  #convert to tensor
  x_train_vgg_axial = tf.convert_to_tensor(x_train_vgg_axial)
  x_valid_vgg_axial = tf.convert_to_tensor(x_valid_vgg_axial)
  x_train_vgg_coronal = tf.convert_to_tensor(x_train_vgg_coronal)
  x_valid_vgg_coronal = tf.convert_to_tensor(x_valid_vgg_coronal)
  x_train_vgg_sagittal = tf.convert_to_tensor(x_train_vgg_sagittal)
  x_valid_vgg_sagittal = tf.convert_to_tensor(x_valid_vgg_sagittal)

  y_train_abnormal = tf.convert_to_tensor(y_train_abnormal)
  y_valid_abnormal = tf.convert_to_tensor(y_valid_abnormal)

  #train seperate models
  #model_axial_abnormal.history = model_axial_abnormal.fit(x_train_vgg_axial,y_train_abnormal,epochs=30,validation_data=(x_valid_vgg_axial,y_valid_abnormal))
  #model_coronal_abnormal.history = model_coronal_abnormal.fit(x_train_vgg_coronal,y_train_abnormal,epochs=30,validation_data=(x_valid_vgg_coronal,y_valid_abnormal))
  #model_sagittal_abnormal.history = model_sagittal_abnormal.fit(x_train_vgg_sagittal,y_train_abnormal,epochs=30,validation_data=(x_valid_vgg_sagittal,y_valid_abnormal))
  
  #train final model
  train_abnormal = model_axial_abnormal.predict(x_train_vgg_axial)
  train_abnormal = np.c_[train_abnormal,model_coronal_abnormal.predict(x_train_vgg_coronal)]
  train_abnormal = np.c_[train_abnormal,model_sagittal_abnormal.predict(x_train_vgg_sagittal)]

  valid_abnormal = model_axial_abnormal.predict(x_valid_vgg_axial)
  valid_abnormal = np.c_[valid_abnormal,model_coronal_abnormal.predict(x_valid_vgg_coronal)]
  valid_abnormal = np.c_[valid_abnormal,model_sagittal_abnormal.predict(x_valid_vgg_sagittal)]

  train_abnormal = tf.convert_to_tensor(train_abnormal)
  valid_abnormal = tf.convert_to_tensor(valid_abnormal)

  model_abnormal.history = model_abnormal.fit(train_abnormal,y_train_abnormal,epochs=30,validation_data=(valid_abnormal,y_valid_abnormal))
  
  free_ram(x_train, x_valid)
  

In [ ]:
#testing the trained model
test_size = 4
abnormal_results = []

for i in range (test_size):
  x_test, y_test_acl, y_test_meniscus, y_test_abnormal = load_cases(False, index=i)

  x_test_vgg_axial = []
  x_test_vgg_coronal = []
  x_test_vgg_sagittal = []
  for case in x_test:
    x_test_vgg_axial.append(tf.reduce_max(VGGmodel.predict(x_test[case]['axial']),0))
    x_test_vgg_coronal.append(tf.reduce_max(VGGmodel.predict(x_test[case]['coronal']),0))
    x_test_vgg_sagittal.append(tf.reduce_max(VGGmodel.predict(x_test[case]['sagittal']),0))

  del x_test

  x_test_vgg_axial = tf.convert_to_tensor(x_test_vgg_axial)
  x_test_vgg_coronal = tf.convert_to_tensor(x_test_vgg_coronal)
  x_test_vgg_sagittal = tf.convert_to_tensor(x_test_vgg_sagittal)

  y_test_abnormal = tf.convert_to_tensor(y_test_abnormal)

  test_abnormal = model_axial_abnormal.predict(x_test_vgg_axial)
  test_abnormal = np.c_[test_abnormal,model_coronal_abnormal.predict(x_test_vgg_coronal)]
  test_abnormal = np.c_[test_abnormal,model_sagittal_abnormal.predict(x_test_vgg_sagittal)]

  test_abnormal = tf.convert_to_tensor(test_abnormal)

  result = model_abnormal.evaluate(test_abnormal, y_test_abnormal)
  abnormal_results.append(result)

print(np.mean(abnormal_results, axis=0))

#Failed attempt 3

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import models
from tensorflow.keras import layers

In [ ]:
# import keras
# from keras.models import Model
# from keras.layers import (
#     Input,
#     Dense,
#     Flatten,
#     Activation,
#     GlobalMaxPooling3D
# )
# from keras.layers.convolutional import (
#     Conv3D,
#     MaxPooling3D,
#     AveragePooling3D
# )
# from keras.layers.normalization import BatchNormalization

### **Building Models**

**use keras built-in Inception V3**

In [ ]:
con_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(256,256,3))
con_base.summary()


inception_model = models.Sequential()

#con_base output_shape = s*6*6*2048
inception_model.add(con_base)

#after appling global average pooling, output_shape = s*2048
inception_model.add(layers.AveragePooling2D(6,6))

con_base.trainable = False

model_axial_abnormal = models.Sequential()
model_axial_abnormal.add(layers.Flatten(input_shape=[1, 1, 2048]))
model_axial_abnormal.add(layers.Dense(2048, activation='relu'))
model_axial_abnormal.add(layers.Dense(1, activation='sigmoid'))

#model_axial_abnormal.summary()

model_coronal_abnormal = models.Sequential()
model_coronal_abnormal.add(layers.Flatten(input_shape=[1, 1, 2048]))
model_coronal_abnormal.add(layers.Dense(2048, activation='relu'))
model_coronal_abnormal.add(layers.Dense(1, activation='sigmoid'))

#model_coronal_abnormal.summary()

model_sagittal_abnormal = models.Sequential()
model_sagittal_abnormal.add(layers.Flatten(input_shape=[1, 1, 2048]))
model_sagittal_abnormal.add(layers.Dense(2048, activation='relu'))
model_sagittal_abnormal.add(layers.Dense(1, activation='sigmoid'))

#model_sagittal_abnormal.summary()

LR_model = models.Sequential()
LR_model.add(layers.Dense(1, activation='sigmoid',input_shape = [3]))

#LR_model.summary()



### **Training models**

**pass to inceptionV3**

In [ ]:
#reduce_max => perform max pooling across slices to obtain a 2048-dimensional vector
x_train_inc_axial = []
x_train_inc_coronal = []
x_train_inc_sagittal = []
for case in x_train:
  x_train_inc_axial.append(tf.reduce_max(inception_model.predict(x_train[case]['axial']),0))
  x_train_inc_coronal.append(tf.reduce_max(inception_model.predict(x_train[case]['coronal']),0))
  x_train_inc_sagittal.append(tf.reduce_max(inception_model.predict(x_train[case]['sagittal']),0))

x_valid_inc_axial = []
x_valid_inc_coronal = []
x_valid_inc_sagittal = []
for case in x_valid:
  x_valid_inc_axial.append(tf.reduce_max(inception_model.predict(x_valid[case]['axial']),0))
  x_valid_inc_coronal.append(tf.reduce_max(inception_model.predict(x_valid[case]['coronal']),0))
  x_valid_inc_sagittal.append(tf.reduce_max(inception_model.predict(x_valid[case]['sagittal']),0))

x_test_inc_axial = []
x_test_inc_coronal = []
x_test_inc_sagittal = []
for case in x_test:
  x_test_inc_axial.append(tf.reduce_max(inception_model.predict(x_test[case]['axial']),0))
  x_test_inc_coronal.append(tf.reduce_max(inception_model.predict(x_test[case]['coronal']),0))
  x_test_inc_sagittal.append(tf.reduce_max(inception_model.predict(x_test[case]['sagittal']),0))

In [ ]:
x_train_inc_axial = tf.convert_to_tensor(x_train_inc_axial)
x_valid_inc_axial = tf.convert_to_tensor(x_valid_inc_axial)
x_test_inc_axial = tf.convert_to_tensor(x_test_inc_axial)

x_train_inc_coronal = tf.convert_to_tensor(x_train_inc_coronal)
x_valid_inc_coronal = tf.convert_to_tensor(x_valid_inc_coronal)
x_test_inc_coronal = tf.convert_to_tensor(x_test_inc_coronal)

x_train_inc_sagittal = tf.convert_to_tensor(x_train_inc_sagittal)
x_valid_inc_sagittal = tf.convert_to_tensor(x_valid_inc_sagittal)
x_test_inc_sagittal = tf.convert_to_tensor(x_test_inc_sagittal)

y_train_abnormal = tf.convert_to_tensor(y_train_abnormal)
y_valid_abnormal = tf.convert_to_tensor(y_valid_abnormal)
y_test_abnormal = tf.convert_to_tensor(y_test_abnormal)
print(x_train_inc_axial[0].shape)

**pass to fully connected layer**

In [ ]:
model_axial_abnormal.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])
model_axial_abnormal.history = model_axial_abnormal.fit(x_train_inc_axial,y_train_abnormal,epochs=30,validation_data=(x_valid_inc_axial,y_valid_abnormal))

In [ ]:
#test accuracy
model_axial_abnormal.predict(x_test_inc_axial,0)

In [ ]:
model_coronal_abnormal.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])
model_coronal_abnormal.history = model_coronal_abnormal.fit(x_train_inc_coronal,y_train_abnormal,epochs=30,validation_data=(x_valid_inc_coronal,y_valid_abnormal))

In [ ]:
#test accuracy
model_coronal_abnormal.predict(x_test_inc_coronal, 0)

In [ ]:
model_sagittal_abnormal.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])
model_sagittal_abnormal.history = model_sagittal_abnormal.fit(x_train_inc_sagittal,y_train_abnormal,epochs=30,validation_data=(x_valid_inc_sagittal,y_valid_abnormal))

In [ ]:
#test accuracy
model_sagittal_abnormal.predict(x_test_inc_sagittal, 0)

**pass to logistic regression**

In [ ]:
axial_inc_prediction_t = model_axial_abnormal.predict(x_train_inc_axial,0)
coronal_inc_prediction_t = model_axial_abnormal.predict(x_train_inc_coronal,0)
sagittal_inc_prediction_t = model_axial_abnormal.predict(x_train_inc_sagittal,0)

LR_train = tf.concat([axial_inc_prediction_t, coronal_inc_prediction_t, sagittal_inc_prediction_t],1)
print(LR_train.shape)

axial_inc_prediction_v = model_axial_abnormal.predict(x_valid_inc_axial,0)
coronal_inc_prediction_v = model_axial_abnormal.predict(x_valid_inc_coronal,0)
sagittal_inc_prediction_v = model_axial_abnormal.predict(x_valid_inc_sagittal,0)

LR_vaild = tf.concat([axial_inc_prediction_v, coronal_inc_prediction_v, sagittal_inc_prediction_v],1)

(18, 3)


In [ ]:
LR_model.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])
LR_model.history = LR_model.fit(LR_train,y_train_abnormal,epochs=60,validation_data=(LR_vaild,y_valid_abnormal))